<a href="https://colab.research.google.com/github/titwan37/GCLRrepo/blob/main/OPCUA_MachineProcessData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-Requisites

In [ ]:
!pip install seaborn
!pip install metpy
!pip install meteocalc
!pip install matplotlib
!pip install mplcursors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for meteocalc: filename=meteocalc-1.1.0-py3-none-any.whl size=8178 sha256=68daaee8578f638c5e63aeed13a7377a5afa5005383f05d43a52b82c2d90f74c
  Stored in directory: /root/.cache/pip/wheels/12/33/7e/c30bba5460b32945c770e9819f74a1e35858ea88864ba145a5
Successfully built meteocalc


In [ ]:
#######################
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mpl_toolkits
import numpy as np
import re
import json
#######################
import seaborn as sns
import mplcursors
#######################
from meteocalc import Temp, dew_point # pip install meteocalc
import datetime as dt
import pandas as pd

## Variables

In [ ]:
#@title Test Variables

from datetime import datetime, timedelta

mode = 1            # 1) from now   # 2) timeReversal
study_period = 120    # days
studyDate = '04.10.2024'  # '2024-08-28'
studyDatetime = '2024-10-02 19:30:00'  # '2024-08-28 19:30:00'

strDateFormat = '%Y-%m-%d'
strDatetimeFormat = '%Y-%m-%d %H:%M:%S'

date_list = []
start_date = None
end_date = None

if (mode==1):
  end_date = datetime.now().date()
  start_date = end_date - timedelta(days=study_period)
  studyDate = end_date.strftime('%Y-%m-%d')
  studyDatetime = (datetime.combine(end_date, datetime.min.time()) + timedelta(hours=19, minutes=30)).strftime(strDatetimeFormat)

  # Calcul range dates
  #date_list = lambda start, end, format: [start + timedelta(days=x) for x in range((end-start).days + 1)]
  date_list = [start_date + timedelta(days=x) for x in range((end_date-start_date).days + 1)]
  #print(dates)

elif (mode==2):
  # Parse the studyDate into a datetime object
  study_date = datetime.strptime(studyDate, '%d.%m.%Y').date()  # previously '%d.%m.%Y' or strDateFormat
  end_date = study_date
  # Calculate the start_date by subtracting the study_period from the study_date
  start_date = end_date - timedelta(days=study_period-1)
  # Calcul range dates
  #date_list = lambda start, end, format: [start + timedelta(days=x) for x in range((end-start).days + 1)]
  date_list = [start_date + timedelta(days=x) for x in range((end_date-start_date).days + 1)]
  #print(dates)

#datefiles = [ '2024-02-01','2024-02-02', '2024-02-20', '2024-02-22', '2024-02-23', '2024-03-11', '2024-03-12']  #, '2024-02-29'
datefiles = [date.strftime(strDateFormat) for date in date_list]
datefile = datefiles[-1]
print(datefiles)

['2024-07-09', '2024-07-10', '2024-07-11', '2024-07-12', '2024-07-13', '2024-07-14', '2024-07-15', '2024-07-16', '2024-07-17', '2024-07-18', '2024-07-19', '2024-07-20', '2024-07-21', '2024-07-22', '2024-07-23', '2024-07-24', '2024-07-25', '2024-07-26', '2024-07-27', '2024-07-28', '2024-07-29', '2024-07-30', '2024-07-31', '2024-08-01', '2024-08-02', '2024-08-03', '2024-08-04', '2024-08-05', '2024-08-06', '2024-08-07', '2024-08-08', '2024-08-09', '2024-08-10', '2024-08-11', '2024-08-12', '2024-08-13', '2024-08-14', '2024-08-15', '2024-08-16', '2024-08-17', '2024-08-18', '2024-08-19', '2024-08-20', '2024-08-21', '2024-08-22', '2024-08-23', '2024-08-24', '2024-08-25', '2024-08-26', '2024-08-27', '2024-08-28', '2024-08-29', '2024-08-30', '2024-08-31', '2024-09-01', '2024-09-02', '2024-09-03', '2024-09-04', '2024-09-05', '2024-09-06', '2024-09-07', '2024-09-08', '2024-09-09', '2024-09-10', '2024-09-11', '2024-09-12', '2024-09-13', '2024-09-14', '2024-09-15', '2024-09-16', '2024-09-17', '2024

In [ ]:
#@title Test Files
import glob, os
sourcefolder = 'C://Users/falempia/Downloads/DropBox/'
sourcefolder = '/content/'

sourcefolder = '/content/drive/MyDrive/CoLab_inbox/'
sourcefolder = 'G://My Drive/CoLab_inbox/'

targetfolder = '/content/drive/MyDrive/Colab_output/'
targetfolder = 'G://My Drive/Colab_output/'

reportfolder = '\\\\Rkamb000282\\cesr\\backup\\'  # 2024-09-30\\2024-09-30_backup7days_Output\\

def lookForNetworkFiles(sourcefolder, folderRadical, datefiles, fileradical):
    # Look for network file according to a fileradical inside a folder and its subfolders named according to decremental formatted days and a folder radical
    gopcuafilenames = [(file, datef, os.path.relpath(os.path.dirname(file), sourcefolder)) for datef in datefiles for file in glob.glob(os.path.join(sourcefolder, datef, '*', folderRadical, datef + fileradical))]
    print(f'FileArray in {sourcefolder} for {fileradical} > {gopcuafilenames}')
    return  gopcuafilenames

def LookForLocalFiles(sourcefolder, datefiles, fileprefix, fileradical):
    # Look for local file according to a fileradical inside a folder and its subfolders named according to decremental formatted days and a folder radical
    #fileradical = '_CF_Global_InOut*.csv'
    # opcuafilenames = [ [sourcefolder + datefile  + '_MES-OPCUA'+ fileradical, datefile] for datefile in datefiles]
    opcuafilenames = [[file, datefile, sourcefolder] for datefile in datefiles for file in glob.glob(sourcefolder + datefile  + fileprefix + fileradical)]
    print(f'FileArray in {sourcefolder} for {fileradical} > {opcuafilenames}')
    return opcuafilenames

def LookForLocalNodeFiles(sourcefolder, datefiles, nodes, fileprefix, fileradical):
    # Look for local file according to a fileradical inside a folder and its subfolders named according to decremental formatted days and a node root and a folder radical
    # 2024-02-20_MES_IMSAPI_node2_CustomFuctions_GlobalValues_s.csv
    #imsapifilenames = [ sourcefolder + datefile + '_MES_IMSAPI_' + node + fileradical for datefile in datefiles for node in nodes]
    # imsapifilenames = [ [sourcefolder + datefile  + '_' + node + '_MES_IMSAPI' + fileradical, datefile, node] for datefile in datefiles for node in nodes]
    imsapifilenames = [[file, datefile, node, sourcefolder] for datefile in datefiles for node in nodes for file in glob.glob(sourcefolder + datefile  + '_' + node + fileprefix + fileradical)]
    print(f'FileArray in {sourcefolder} for {fileradical} > {imsapifilenames}')
    return imsapifilenames

def LookAlternatively(sourcefolder, reportfolder, datefiles, folderRadical, fileradical):
    arr1 = []
    arr2 = []
    # To check if the reportfolder on the network is available
    if os.path.isdir(reportfolder):
        print(f"Report folder '{reportfolder}' is available on the network.")
        arr1 = lookForNetworkFiles(reportfolder, folderRadical, datefiles, fileradical)
    else:
        print(f"Report folder '{reportfolder}' is not available on the network.")
        arr2 = LookForLocalFiles(sourcefolder, datefiles, '', fileradical)
        print(f"Source folder '{sourcefolder}' is available on the cloud.")
    array_all = arr1 + arr2
    #print(f"Combined file array: {array_all}")
    return  array_all


folderRadical = 'OPCUA-CustomFunctions-Study'
fileradical = '_MES-OPCUA_CF_GlobalValues_s.csv'
#gopcuafilenames = lookForNetworkFiles(reportfolder, folderRadical, datefiles, fileradical)
gopcuafilenames = LookAlternatively(sourcefolder, reportfolder, datefiles, folderRadical, fileradical)

fileradical = '_MES-OPCUA_CF_Global_UploadProcessData_s.csv'
#processfilename = folder + datefile + processfilenameRadical
processfilenames = LookAlternatively(sourcefolder, reportfolder, datefiles, folderRadical, fileradical)

Report folder '\\Rkamb000282\cesr\backup\' is not available on the network.
FileArray in G://My Drive/CoLab_inbox/ for _MES-OPCUA_CF_GlobalValues_s.csv > []
Source folder 'G://My Drive/CoLab_inbox/' is available on the cloud.
Report folder '\\Rkamb000282\cesr\backup\' is not available on the network.
FileArray in G://My Drive/CoLab_inbox/ for _MES-OPCUA_CF_Global_UploadProcessData_s.csv > []
Source folder 'G://My Drive/CoLab_inbox/' is available on the cloud.


In [ ]:
filename = '//rkamb000282/CESR/backup/2024_01_11/2024_01_11_backup4days_Output/OPCUA-CustomFunctions-Study/2024-12-22_MES-OPCUA_CF_Global_UploadProcessData_s.csv'
folder = 'C://Users/falempia/Downloads/DropBox/'
folder = 'C://CESR/DropBox/ProcessData/'
folder = '/content/'
datefile = datefiles[-1]
nodes = ['node1', 'node2' ]

# 2024-02-20_MES_IMSAPI_node2_CustomFuctions_GlobalValues_s.csv
#imsapifilenames = [ folder + datefile + '_MES_IMSAPI_' + node + fileradical for datefile in datefiles for node in nodes]
#imsapifilenames = [ folder + datefile  + '_' + node + '_MES_IMSAPI' + fileradical for datefile in datefiles for node in nodes]

processfilenameRadical = '_MES-OPCUA_CF_Global_UploadProcessData_s.csv'
#processfilename = folder + datefile + processfilenameRadical
processfilenames = [ folder + datefile  + processfilenameRadical for datefile in datefiles for node in nodes]
print(processfilenames)

processfilename = processfilenames[0]
print(processfilename)

setupfilenameRadical = '_MES-OPCUA_CF_Global_Setups_s.csv'
#setupfilename = folder + datefile + setupfilenameRadical
setupfilenames = [ folder + datefile  + setupfilenameRadical for datefile in datefiles for node in nodes]
setupfilename = [0]

colprocessfilename = folder + datefile +'_MES-OPCUA_CF_UploadProcessData_col.csv'
colsetupfilename = folder + datefile +'_MES-OPCUA_CF_Global_Setups_col.csv'
pngsetupfilename = folder + datefile +'_MES-OPCUA_CF_Global_Setups_chart_'

['/content/2024-07-09_MES-OPCUA_CF_Global_UploadProcessData_s.csv', '/content/2024-07-09_MES-OPCUA_CF_Global_UploadProcessData_s.csv', '/content/2024-07-10_MES-OPCUA_CF_Global_UploadProcessData_s.csv', '/content/2024-07-10_MES-OPCUA_CF_Global_UploadProcessData_s.csv', '/content/2024-07-11_MES-OPCUA_CF_Global_UploadProcessData_s.csv', '/content/2024-07-11_MES-OPCUA_CF_Global_UploadProcessData_s.csv', '/content/2024-07-12_MES-OPCUA_CF_Global_UploadProcessData_s.csv', '/content/2024-07-12_MES-OPCUA_CF_Global_UploadProcessData_s.csv', '/content/2024-07-13_MES-OPCUA_CF_Global_UploadProcessData_s.csv', '/content/2024-07-13_MES-OPCUA_CF_Global_UploadProcessData_s.csv', '/content/2024-07-14_MES-OPCUA_CF_Global_UploadProcessData_s.csv', '/content/2024-07-14_MES-OPCUA_CF_Global_UploadProcessData_s.csv', '/content/2024-07-15_MES-OPCUA_CF_Global_UploadProcessData_s.csv', '/content/2024-07-15_MES-OPCUA_CF_Global_UploadProcessData_s.csv', '/content/2024-07-16_MES-OPCUA_CF_Global_UploadProcessData_s.

In [ ]:
def getShapeSize(label='', df=None):
  # Assuming you have a DataFrame called 'df'
  dimensions = df.shape
  num_rows = dimensions[0]
  num_columns = dimensions[1]
  print(f"Dataframe={label} has Number of rows: {num_rows} and columns : {num_columns}")

def getColumnNames(dnp):
  print ('############')
  cols = dnp.columns.values.tolist()
  for col in range(len(cols)): print(f'index ({col}) -> {cols[col]}')

# Import dataset

In [ ]:
import numpy as np
import pandas as pd
def importDataset(filename):
  dataset = pd.DataFrame()
  try:
    dataset = pd.read_csv(filename, sep=';', on_bad_lines='skip', low_memory=False)
      # Continue with your data processing
    headers = dataset.columns
    #print(headers)
  except FileNotFoundError:
      print(f"{filename} File not found. Please check the file path and try again.")

  except pd.errors.ParserError as e:
      print(f"Error parsing CSV file: {e}")
  return dataset

def printFirstRow(dataset):
  first_row = dataset.iloc[0]
  print(first_row)

def getShapeSize(label='', df=None):
  # Assuming you have a DataFrame called 'df'
  dimensions = df.shape
  num_rows = dimensions[0]
  num_columns = dimensions[1]
  print(f"Dataframe={label} has Number of rows: {num_rows} and columns : {num_columns}")
  return dimensions

def getColumnNames(dnp):
  print ('############')
  cols = dnp.columns.values.tolist()
  for col in range(len(cols)): print(f'index ({col}) -> {cols[col]}')

# node1_df = importDataset(imsapifilenames[0])
# node2_df = importDataset(imsapifilenames[1])
# getShapeSize('node1_df', node1_df)
# #getColumnNames(node1_df)
# getShapeSize('node2_df',node2_df)
# #getColumnNames(node2_df)

def aggregateDataFrame(imsapifilenames):
  nodes_df = []
  for imsapifilename in imsapifilenames:
    nodeZ_df = importDataset(imsapifilename)
    dim = getShapeSize(imsapifilename, nodeZ_df)
    if (dim[0]>0):
      nodes_df.append(nodeZ_df)
    else:
      print(f'Expected File:\n{imsapifilename}')
      #getFolderFileList(folder)
  aggregated_df = pd.DataFrame()
  print (type(nodes_df))
  if len(nodes_df)>0 :
    aggregated_df = pd.concat(nodes_df, axis=0)
    aggregated_df = aggregated_df.drop_duplicates() # remove duplicates
  print (type(aggregated_df))
  getShapeSize('All aggregated',aggregated_df)
  return aggregated_df

processDataset = aggregateDataFrame(processfilenames)
setupDataset = aggregateDataFrame(setupfilenames)

/content/2024-07-09_MES-OPCUA_CF_Global_UploadProcessData_s.csv File not found. Please check the file path and try again.
Dataframe=/content/2024-07-09_MES-OPCUA_CF_Global_UploadProcessData_s.csv has Number of rows: 0 and columns : 0
Expected File:
/content/2024-07-09_MES-OPCUA_CF_Global_UploadProcessData_s.csv
/content/2024-07-09_MES-OPCUA_CF_Global_UploadProcessData_s.csv File not found. Please check the file path and try again.
Dataframe=/content/2024-07-09_MES-OPCUA_CF_Global_UploadProcessData_s.csv has Number of rows: 0 and columns : 0
Expected File:
/content/2024-07-09_MES-OPCUA_CF_Global_UploadProcessData_s.csv
/content/2024-07-10_MES-OPCUA_CF_Global_UploadProcessData_s.csv File not found. Please check the file path and try again.
Dataframe=/content/2024-07-10_MES-OPCUA_CF_Global_UploadProcessData_s.csv has Number of rows: 0 and columns : 0
Expected File:
/content/2024-07-10_MES-OPCUA_CF_Global_UploadProcessData_s.csv
/content/2024-07-10_MES-OPCUA_CF_Global_UploadProcessData_s.c

# Extract and Isolate Process dataset

In [ ]:
def addDateTimeColumn(df, timestampString, datetime):
  try:
    # Add a new column 'Row Number' with sequentially numbered rows
    #df['rowIndex'] = range(1, len(df) + 1)
    df.insert(0, 'rowIndex', range(1, len(df) + 1))
    # Assuming you have a DataFrame named 'df' with a column named 'datetime' containing timestamp strings
    df.insert(0, datetime, pd.to_datetime(df[timestampString], format='mixed'))
  except Exception as e:
    print("An error occurred:", str(e))
  finally:
    return df

def redirectParsingValuesToColumns(df, columnsToBeSet, columnsOfValues ):
  nbColTBS = len(columnsToBeSet)
  nbColVal = len(columnsOfValues)
  for i in range(nbColVal):  # Corrected the range function
    colNameTBset = columnsToBeSet[i]
    colNameValue = columnsOfValues[i]
    if colNameValue in df.columns:
      # Rows that contain the 'C1|ActualTemp' column --> detect double chambers
      rows_with_c1_actual_indices = df[colNameValue].notnull()
      # Assign the value of 'C1|ActualTemp' to 'rActualTemperature1' in the rows where 'C1|ActualTemp' exists
      df.loc[rows_with_c1_actual_indices, colNameTBset] = df[colNameValue]
  return df

def setSpecialParsingDoubleChambers(df):
  # Check if the 'C1|ActualTemp' column exists in the DataFrame
  print (df.columns)
  if ' C1|ActualTemp' in df.columns:
      columnsToBeSet =  ['rActualTemperature1','ActualHumidity1', 'rActualTemperature2','ActualHumidity2']
      columnsOfValues = [' C1|ActualTemp', ' C1|ActualHumidity', ' C2|ActualTemp', ' C2|ActualHumidity']
      # redirect Parsing
      df = redirectParsingValuesToColumns(df, columnsToBeSet, columnsOfValues)
  else:
      print("The 'C1|ActualTemp' column does not exist in the DataFrame.")
  return df

def extractProcessDataLight(dataset, filename):
  # Assuming you have a DataFrame called 'dataset'
  getShapeSize('dataset', dataset)
  # Convert the 'timestampString' to a pandas DatetimeIndex
  if 'timestampString' in dataset.columns:
    dataset['datetime'] = pd.to_datetime(dataset['timestampString'], format='mixed')
  else:
    print("The 'timestampString' column does not exist in the DataFrame.")
  # Split the 'inArgs' column by ',' and create a new DataFrame with the split values
  if 'inArgs' in dataset.columns:
    split_values = dataset['inArgs'].str.split(',', expand=True)

  # Create a new DataFrame with the split values and the datetime index
  df = pd.concat([dataset['datetime'], dataset['machineName'], dataset['stationDescription'], split_values], axis=1)
  # Remove duplicate rows based on the datetime index
  df = df[~df.duplicated(subset=['datetime'])]
  # parsing double chamber
  df = setSpecialParsingDoubleChambers(df)

  # Save the new DataFrame as a CSV file
  df.to_csv(filename, index=False)  # Save the new DataFrame as a CSV file
  print(f'Save concat splitted values under > {filename}')
  # Assuming you have a DataFrame called 'df'
  dimensions = df.shape
  num_rows = dimensions[0]
  num_columns = dimensions[1]
  print(f"Number of rows: {num_rows} and columns : {num_columns}")
  return df

def extractProcessData(dataset, filename):
  # Assuming you have a DataFrame called 'df'
  column_data1 = dataset['timestampString']
  column_data2a = dataset['machineName']
  column_data2b = dataset['stationDescription']
  column_data3 = dataset['inArgs']
  # Convert the column_data1 to a pandas DatetimeIndex:
  datetime_index = pd.to_datetime(column_data1)
  # Assuming you have a column_data variable containing the desired column data
  column_data_list = column_data3.astype(str).tolist()  # Convert column_data to a list of strings
  split_values = [row.split(',') for row in column_data_list]  # Split each string by comma
  df = pd.DataFrame(split_values, index=datetime_index)  # Create a new DataFrame
  # Add the datetime column and 'stationDescription' column
  df = addDateTimeColumn(df, 'timestampString', 'datetime')
  #df.insert(0, 'stationDescription', column_data2b)
  #df.insert(0, 'machineName', column_data2a)
  # Drop rows with duplicate labels
  df = df[~df.index.duplicated(keep='first')]

  # Reset the index of the new DataFrame
  df.reset_index(drop=True, inplace=True)

  # Save the new DataFrame as a CSV file
  df.to_csv(filename, index=False)  # Save the new DataFrame as a CSV file
  # Assuming you have a DataFrame called 'df'
  dimensions = df.shape
  num_rows = dimensions[0]
  num_columns = dimensions[1]
  print(f"Number of rows: {num_rows} and columns : {num_columns}")
  return df

purprocessfilename  = str(colprocessfilename[:len(colprocessfilename)-7]) + 'pur.csv'
#process_df = extractProcessData(processDataset, purprocessfilename)
getShapeSize('processDataset', processDataset)
process_df = extractProcessDataLight(processDataset, purprocessfilename)
getShapeSize('process_df', process_df)

Dataframe=processDataset has Number of rows: 0 and columns : 0
Dataframe=dataset has Number of rows: 0 and columns : 0
The 'timestampString' column does not exist in the DataFrame.


KeyError: 'datetime'

In [ ]:
process_machine_0 = process_df['machineName'].unique()
print (f'Number of known machines : {len(process_machine_0)}')
print (process_machine_0)

In [ ]:
def extractProcessDataFromMachine(data, machineName, filename):
  # Filter the rows where machineName is 'TPS_DPU_313'
  machineName_filtered_data = data[data['machineName'] == machineName]
  # Save the filtered data to a CSV file
  machineName_filtered_data.to_csv(filename, index=False)
  print(f'extracted data from machine {machineName} are saved under {filename}')

machineName = 'IST'
machineName_processfilename  = str(colprocessfilename[:len(colprocessfilename)-7]) + machineName  +'.csv'
extractProcessDataFromMachine(process_df, machineName, machineName_processfilename)

## Extract and Isolate Setup dataset

In [ ]:
def extractSetupDataSuperLight(dataset, filename):
  # Convert the 'timestampString' to a pandas DatetimeIndex
  dataset['datetime'] = pd.to_datetime(dataset['timestampString'])
  datetime_index =  dataset['datetime']
  if not dataset.index.is_unique:
    dataset.reset_index(drop=True, inplace=True)  # Reset the index to create a new unique index
  # Split each string in the 'inArgs' column by JSON format and create a new DataFrame with the split values
  #json_values = [json.loads(data) for data in dataset['inArgs']]
  json_values = [json_data for json_data in dataset['inArgs']]
  json_df = pd.DataFrame(json_values)
  # Set a unique index on json_df
  json_df.index = range(len(json_df))
  # Reset the index of json_df
  json_df.reset_index(drop=True, inplace=True)
  # Create a new DataFrame with the split values and the datetime index
  df = pd.concat([dataset, json_df], axis=1)
  # Set the index to the 'datetime' column
  df.set_index('datetime', inplace=True)
  # Drop all duplicate rows based on the index
  df = df[~df.index.duplicated()]
  # Add the 'timestampString', 'machineName', 'stationDescription', and 'methodName' columns to the new DataFrame
  #df['machineName'] = dataset['machineName']
  #df['stationDescription'] = dataset['stationDescription']
  #df['methodName'] = dataset['methodName']
  # Drop all duplicate rows based on the index
  df = df[~df.duplicated()]
  # Save the new DataFrame as a CSV file
  df.to_csv(filename, index=False)  # Save the new DataFrame as a CSV file
  # Print the shape of the new DataFrame
  getShapeSize('InArgs with Machine and Timestamps', df)

  # Method 1: Using boolean indexing
  filtered_df = df[df['methodName'] == 'cfRocheCESR.cfpSetup']
  #filtered_df.to_csv(filename, index=False)  # Save the new DataFrame as a CSV file
  # Assuming you have a DataFrame called 'df'
  getShapeSize('only cfRocheCESR.cfpSetup', filtered_df)
  return df

def extractSetupDataLight(dataset, filename):
  # Convert the 'timestampString' to a pandas DatetimeIndex
  dataset['datetime'] = pd.to_datetime(dataset['timestampString'])
  # Split each string in the 'inArgs' column by JSON format and create a new DataFrame with the split values
  #json_values = [json.loads(data) for data in dataset['inArgs']]
  json_values = [(json_data) for json_data in dataset['inArgs']]

  df = pd.DataFrame(json_values)

  # Add the 'timestampString', 'machineName', 'stationDescription', and 'methodName' columns to the new DataFrame
  df.insert(0, 'methodName', dataset['methodName'])
  df.insert(0, 'stationDescription', dataset['stationDescription'])
  df.insert(0, 'machineName', dataset['machineName'])
  df.insert(0, 'datetime', dataset['datetime'])
  # Remove duplicate rows based on the datetime index
  df = df[~df.duplicated(subset=['datetime'])]
  # Save the new DataFrame as a CSV file
  df.to_csv(filename, index=False)
  # Print the shape of the new DataFrame
  getShapeSize('InArgs with Machine and Timestamps', df)

  # Method 1: Using boolean indexing
  filtered_df = df[df['methodName'] == 'cfRocheCESR.cfpSetup']
  filtered_df.to_csv(filename, index=False)  # Save the new DataFrame as a CSV file
  # Assuming you have a DataFrame called 'df'
  getShapeSize('only cfRocheCESR.cfpSetup', filtered_df)
  return df

#from os import getsid
def extractSetupData(dataset, filename):
  # Assuming you have a DataFrame called 'df'
  column_data1 = dataset['timestampString']
  column_data2a = dataset['machineName']
  column_data2b = dataset['stationDescription']
  column_data3 = dataset['methodName']
  column_data4 = dataset['inArgs']

  # Assuming you have a column_data variable containing the desired column data
  column_data_list = column_data4.astype(str).tolist()  # Convert column_data to a list of strings
  json_values = [ (json_data) for json_data in column_data_list]  # Split each string by json format

  # Create a new DataFrame
  df = pd.DataFrame(json_values)
  df.insert(0, 'methodName', column_data3)
  df.insert(0, 'stationDescription', column_data2b)
  df.insert(0, 'machineName', column_data2a)
  df.insert(0, 'timestampString', column_data1)
  getShapeSize('InArgs with Machine and Timestamps', df)
  df.to_csv(filename, index=False)
  # Method 1: Using boolean indexing
  filtered_df = df[df['methodName'] == 'cfRocheCESR.cfpSetup']
  #filtered_df.to_csv(filename, index=False)  # Save the new DataFrame as a CSV file
  # Assuming you have a DataFrame called 'df'
  getShapeSize('only cfRocheCESR.cfpSetup', filtered_df)
  return filtered_df

pursetupfilename  = colsetupfilename[:len(colsetupfilename)-5] + 'pur.csv'
#setup_df = extractSetupData(setupDataset, pursetupfilename)
setup_df = extractSetupDataSuperLight(setupDataset, pursetupfilename)

# Data Calculation

## Dew Point Calculation

In [ ]:
import math
from meteocalc import Temp, dew_point # pip install meteocalc
# https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.saturation_vapor_pressure.html

from metpy.calc import saturation_vapor_pressure # pip install metpy
from metpy.calc import dewpoint_from_relative_humidity # pip install metpy
from metpy.units import units

const=dict(
  K1=6.113,
  K2=17.625,
  K3=243.12
)

def iTACDewPoint(t, rH, const):
  # Calculate the Taupunkt (`C) für iTAC
  adp1 = (const['K2']*T / (const['K3']+T)) + math.log(rH/100.)
  adp2 = (const['K2']*const['K3'] / (const['K3']+T)) - math.log(rH/100.)
  tp = const['K3'] * (adp1/adp2)
  return tp

def calculateDewPoint(T, rH, dp_itac, index=0):
  print(f'Temperature of Sensor {index} ={T} (`C)')
  print(f'relative Humidity {index}     ={rH} (%)')

  # Calculate the dew point von Arden Buck
  dp = dew_point(temperature=T, humidity=rH)
  print(f'Dew Point Equation von Arden Buck ={dp}(`C)')

  # Calculate the dew point von National Science Foundation
  nsf_dewP = dewpoint_from_relative_humidity(T * units.degC, rH* units.percent)
  print(f'Dew Point Equation von National Science Foundation ={nsf_dewP}(`C)')

  # Calculate the Saturation Vapor Pressure von National Science Foundation
  nsf_Pvs = saturation_vapor_pressure(T * units.degC).to('hPa')
  print(f'Saturation Vapor Pressure von National Science Foundation = {nsf_Pvs}  (hPa)')

  # Calculate the Saturation Vapor Pressure
  Ps = const['K1'] * math.exp( const['K2']*T / (const['K3']+T)  )
  print(f'Saturation Vapor Pressure = {Ps}  (hPa)')
  # Calculate the Actual Vapor Pressure
  Pa = rH / 100. * Ps
  print(f'Actual Vapor Pressure = {Pa} (hPa)')

  # Calculate the Taupunkt (`C) für iTAC
  adp1 = (const['K2']*T / (const['K3']+T)) + math.log(rH/100.)
  adp2 = (const['K2']*const['K3'] / (const['K3']+T)) - math.log(rH/100.)
  tp = const['K3'] * (adp1/adp2)
  tpt = "%.2f" % tp
  tpr = round(tp, 2)
  print(f'Taupunkt from Excel für iTAC  = {tp} (`C)')
  print(f'Taupunkt rounded für iTAC  = {tpr} (`C)')
  print(f'==================================================')
  print(f'Dew Point from iTAC MES Suite  = {dp_itac} (`C)')
  if (dp_itac==tpr):
      testResult = 'PASSED'
      print(f'TEST {testResult} ')
  else :
      testResult = 'FAILED'
      print(f'TEST {testResult}, expected {tpr} <> current {dp_itac} ')
  print(f'==================================================\n')

def verify(Sensors):
  for sensor in Sensors:
    print(sensor)
    #print(sensor['T1'])
    calculateDewPoint(sensor['T1'], sensor['rH1'], sensor['dp1'], 1)
    calculateDewPoint(sensor['T2'], sensor['rH2'], sensor['dp2'], 2)

# relative Humidity (%)
rH = 44.579998
# temperature in degree (`C)
T = 22.039999

Sensors=(
          dict(rH1= 44.41, T1=22.15, dp1=9.45, rH2=45.5, T2=22, dp2=9.68),
          dict(rH1= 44.489998, T1=22.13, dp1=9.46, rH2=45.5, T2=22.01, dp2=9.69),
          dict(rH1= 44.66, T1=22.14, dp1=9.53, rH2=45.5, T2=22.06, dp2=9.73),
          dict(rH1= 44.579998, T1=22.15, dp1=9.51, rH2=45.5, T2=22.039999, dp2=9.72),
          dict(rH1= 44.739998, T1=22.15, dp1=9.56, rH2=45.48, T2=22.09, dp2=9.76),
          dict(rH1= 45, T1=24, dp1=10.0, rH2=85, T2=45, dp2=-5.0)
          )


verify(Sensors)

# Data Modelization

## Process Data Modelization

In [ ]:
class DewPointMeasurementDetails:
  def __init__(self, row=None) -> None:
    self.timestampString = ''
    self.machineName = ''
    self.stationNB = ''
    self.stationDescription=''
    self.rActualTemperature1= -1.1
    self.rActualTemperature2= -1.1
    self.dewPoint1= -1.11
    self.dewPoint2= -1.11
    self.ActualHumidity1= -1.1
    self.ActualHumidity2= -1.1
    self.rActCurrent_X_Axis = -1.1
    self.rActCurrent_Z_Axis = -1.1
    self.outOfRange = False
    # Insert data
    if (len(row)>0): self.Parse(row)
    pass

  def Parse(self, row):
    # 3005001060018026505, Temperature, 24.15, c, Humidity, 43.33, %
    if (len(row)>8):
      #print(row)
      self.timestampString = str(row[0]).strip()
      self.machineName = row[1]  #.strip() -> exception
      self.stationDescription = row[2]  #.strip() -> exception
      self.stationNB = str(row[3]).strip()

      # --> Handle TPS IST
      if (str(row[4]).lower().strip()=='Temperature'.lower() or str(row[4]).lower().strip()=='Temperatute'.lower()):
        self.rActualTemperature1 = float(str(row[5]).strip())
      if (str(row[7]).lower().strip()=='Humidity'.lower()):
        self.ActualHumidity1 = float(str(row[8]).strip())

      #print(f'machineName={self.machineName}, number of rows={len(row)})
      #if (self.machineName=='KLS_103'):
      #  print(f'machine {self.machineName}')

    # 30010230600180206, rActualTemperature1, 24.97, C, rActualTemperature2, 24.939999, C, ActualHumidity1, 11.57, %, ActualHumidity2, 11.679999, %, PositionX, 222.09274, mm, PositionZ, -860.79047, mm, TargetAmountSensors, 2, Sensors, ActualAmountSensors, 2, Sensors, SensorShutterOpen, false, BOOL, SensorShutterClosed, true, BOOL, SensorTurnUnit_0, true, BOOL, SensorTurnUnit_90, false, BOOL, InletSeg_ConvOn, false, BOOL, TurnSeg_ConvForwOn, false, BOOL, TurnSeg_ConvBackOn, false, BOOL, OutletSeg_ConvOn, false, BOOL, IntraSeg_ConvForwOn, false, BOOL, IntraSeg_ConvBackOn, false, BOOL, ShuttleSeg_ConvForwOn, false, BOOL, ShuttleSeg_ConvBackOn, false, BOOL, BufferMag1_2_MotorFwOn, false, BOOL, BufferMag1_2_MotorBwOn, false, BOOL, BufferMag3_4_MotorFwOn, false, BOOL, BufferMag3_4_MotorBwOn, false, BOOL, strSNR_Probe1, 0020494638, U, strSNR_Probe2, 0020494642, U, rActCurrent_X_Axis, 0, U, rActCurrent_Z_Axis, 0, U
    if (len(row)>14):
      # legend -> [4]
      if (str(row[4]).strip()=='rActualTemperature1'):   # --> Handle Klimaschrank Mod Typ 1
        self.rActualTemperature1 = float(row[5].strip())
      elif (str(row[4]).strip()=='ActualTemp'):           # --> Handle Klimaschrank Ext
        self.rActualTemperature1 = float(row[5].strip())
      elif (str(row[4]).strip()=='ActualTemp1'):           # --> Handle Klimaschrank KLS_311
        self.rActualTemperature1 = float(row[5].strip())
      elif (str(row[4]).strip()=='ActualTemp_1'):           # --> Handle Klimaschrank KLS_103
        self.rActualTemperature1 = float(row[5].strip())
      elif (str(row[4]).strip()=='C1|ActualTemp'):   # --> Handle Klimaschrank Ext Dbl Chamber
        self.rActualTemperature1 = float(row[5].strip())
      # unit -> [6]
      # legend -> [7]
      if (str(row[7]).strip()=='rActualTemperature2'):
        self.rActualTemperature2 = float(row[8].strip())
      elif (str(row[7]).strip()=='ActualTemp2'):           # --> Handle Klimaschrank 311
        self.rActualTemperature1 = float(row[8].strip())
      elif (str(row[7]).strip()=='ActualHumidity'):
        self.ActualHumidity1 = float(row[8].strip())
      elif (str(row[7]).strip()=='ActualHumidity1'):
        self.ActualHumidity1 = float(row[8].strip())
      elif (str(row[7]).strip()=='C1|ActualHumidity'):   # --> Handle Klimaschrank Ext Dbl Chamber
        self.ActualHumidity1 = float(row[8].strip())
      # unit -> [9]
      # legend -> [10]
      if (str(row[10]).strip()=='ActualHumidity1'):      # --> Handle Klimaschrank Mod Typ 1
        self.ActualHumidity1 = float(row[11].strip())
      # unit -> [12]
      # legend -> [13]
      if (str(row[13]).strip()=='ActualHumidity2'):   # --> Handle Klimaschrank Mod Typ 1
        self.ActualHumidity2 = float(row[14].strip())

    if (len(row)>20):
      if (str(row[16]).strip()=='PositionX'):   # --> Handle Klimaschrank Mod Typ 1
        self.rActCurrent_X_Axis = float(row[17].strip())
      if (str(row[19]).strip()=='PositionZ'):   # --> Handle Klimaschrank Mod Typ 1
        self.rActCurrent_Z_Axis = float(row[20].strip())

    if(len(row)>29):
      if (str(row[25]).strip()=='C2|ActualTemp'):   # --> Handle Klimaschrank Ext Dbl Chamber
        self.rActualTemperature2 = float(row[26].strip())
      elif (str(row[28]).strip()=='C2|ActualHumidity'):   # --> Handle Klimaschrank Ext Dbl Chamber
        self.ActualHumidity2 = float(row[29].strip())

    if(len(row)>47):
      #print(str(row[46]).strip())
      if (str(row[46]).strip()=='AverageTemperature'):   # --> Handle Magazinlager
        self.rActualTemperature1 = float(row[47].strip())
      if (str(row[43]).strip()=='AverageHumidity'):
        self.ActualHumidity1 = float(row[44].strip())

    if(len(row)>86):
      # rActCurrent_X_Axis
      if (str(row[82]).strip()=='rActCurrent_X_Axis'):   # --> Handle X Y Z Axis
        self.rActCurrent_X_Axis = float(row[83].strip())
      if (str(row[85]).strip()=='rActCurrent_Z_Axis'):
        self.rActCurrent_Z_Axis = float(row[86].strip())

    if (self.rActualTemperature1>0 and self.ActualHumidity1>0):
      self.dewPoint1 = self.calculateDewPoint(self.rActualTemperature1, self.ActualHumidity1)

    if (self.rActualTemperature2>0 and self.ActualHumidity2>0):
      self.dewPoint2 = self.calculateDewPoint(self.rActualTemperature2, self.ActualHumidity2)


  def calculateDewPoint(self, temp, hygro):
    # Define the temperature and relative humidity
    temperature = Temp(temp, 'c')  # Replace 25 with your actual temperature in Celsius
    humidity = hygro  # Replace 60 with your actual relative humidity
    # Calculate the dew point
    #dp = dew_point(temperature=temperature, humidity=humidity)
    dp = iTACDewPoint(temperature, humidity, const)
    # Print the dew point in Celsius and Fahrenheit
    # print('Dew Point in Celsius:', dp.c)
    # print('Dew Point in Fahrenheit:', dp.f)
    return dp
    #return dp.c

  def isOutOfRange(self):
    if (self.rActualTemperature1>200):
      self.outOfRange = True
      print(f'{self.timestampString} {self.machineName} recorded T1= {self.rActualTemperature1} (`C) & rH1= {self.ActualHumidity1} (%) at {self.stationDescription}')
      return True
    if (self.rActualTemperature2>200):
      self.outOfRange = True
      print(f'{self.timestampString} {self.machineName} recorded T2= {self.rActualTemperature2} (`C) & rH2= {self.ActualHumidity2} (%) at {self.stationDescription}')
      return True
    if (self.ActualHumidity1>100):
      self.outOfRange = True
      print(f'{self.timestampString} {self.machineName} recorded T1= {self.rActualTemperature1} (`C) & rH1= {self.ActualHumidity1} (%) at {self.stationDescription}')
      return True
    if (self.ActualHumidity2>100):
      self.outOfRange = True
      print(f'{self.timestampString} {self.machineName} recorded T2= {self.rActualTemperature2} (`C) & rH2= {self.ActualHumidity2} (%) at {self.stationDescription}')
      return True
    if (self.rActualTemperature1==0 and self.ActualHumidity1==0):
      self.outOfRange = True
      #print(f'{self.timestampString} {self.machineName} recorded T1= {self.rActualTemperature1} (`C) & rH1= {self.ActualHumidity1} (%) at {self.stationDescription}')
      return True
    return  False

  def isValid(self):
    #return (self.rActualTemperature1==0 and self.ActualHumidity1==0)
    return (self.rActualTemperature1>-1 and self.ActualHumidity1>-1) # and

  @staticmethod
  def get_Columns():
    return ['timestampString', 'machineName', 'ActualTemperature1', 'ActualHumidity1', 'DewPoint1', 'ActualTemperature2', 'ActualHumidity2', 'DewPoint2', 'stationNumber', 'stationDescription', 'rActCurrent_X_Axis', 'rActCurrent_Z_Axis']
  @staticmethod
  def get_Headers():
    return DewPointMeasurementDetails.get_Columns().join(';')

  def to_csv(self):
    return [self.timestampString, self.machineName, self.rActualTemperature1, self.ActualHumidity1, self.dewPoint1, self.rActualTemperature2, self.ActualHumidity2, self.dewPoint2, self.stationNB, self.stationDescription, self.rActCurrent_X_Axis, self.rActCurrent_Z_Axis]

  def __str__(self):
    return f"Time={self.timestampString}, Machine={self.machineName}, T1={self.rActualTemperature1}, H1={self.ActualHumidity1}, T2={self.rActualTemperature2}, H2={self.ActualHumidity2}, ID={self.stationNB}, Name={self.stationDescription}"

## Setup Data Modelization

In [ ]:
import json

class cfpSetupSettings:
  def __init__(self, row=None) -> None:
    #self.datetime = None
    self.index = 0
    self.timestampString = ''
    self.machineName = ''
    self.stationNB = ''
    self.stationDescription=''
    self.json_data = ''
    self.SetupType = ''
    self.WorkOrderNumber= ''
    self.WorkstepAvo= ''
    self.WorkStepNumber= ''
    self.StationNumber= ''
    self.iOperationMode= ''
    self.strJobNumber= ''
    self.strRecipe= ''
    self.strFormat= ''
    self.strProcessStep= ''
    self.strJobNumber= ''
    self.iFanState= -1.1
    self.iFanSpeed= -1.1
    self.iDwelTime= ''
    self.iPlateCount= ''
    self.iCleanState= ''
    self.Chamber1TempSet= -1.1
    self.Chamber1ToleranceTempMin= -1.1
    self.Chamber1ToleranceTempMax = -1.1
    self.Chamber1HumiditySet = -1.1
    self.Chamber1ToleranceHumidityMin = -1.1
    self.Chamber1ToleranceHumidityMax = -1.1
    ###
    self.Chamber1TempSDMax = -1.1
    self.Chamber1TempSDMin = -1.1
    ###
    self.Chamber1HumiditySDMax = -1.1
    self.Chamber1HumiditySDMin = -1.1
    ###
    self.Chamber1SettlingTimeSet = -1.1
    self.Chamber1ConveyorVeloSet = -1.1
    self.DeviationTimeRange = -1.1
    ###
    self.SensorDriftTempWarnLimit = -1.1
    self.SensorDriftTempAlarmLimit = -1.1
    self.SensorDriftHumWarnLimit = -1.1
    self.SensorDriftHumAlarmLimit = -1.1
    # Insert data
    if (len(row)>0): self.Parse(row)
  pass
# 2024-01-30 09:43:51.385,KLS_107,Klimaschrank Ext (KLS107),cfRocheCESR.cfpSetup,"{""Info"":{""datatype"":""String"",""value"":{""WorkOrderNumber"":{""datatype"":""String"",""value"":""11138069""},""WorkstepAvo"":{""datatype"":""Short"",""value"":1691000000},""WorkStepNumber"":{""datatype"":""Short"",""value"":78}}},""StationNumber"":{""datatype"":""String"",""value"":""30010200600180202""},""strJobNumber"":{""datatype"":""String"",""value"":""11138069""},""iOperationMode"":{""datatype"":""Short"",""value"":7},""strFormat"":{""datatype"":""String"",""value"":""09539158001""},""strProcessStep"":{""datatype"":""String"",""value"":""BGDeckmembran""},""partChamber"":{""datatype"":""String"",""value"":""1""},""Chamber1TempSet"":{""datatype"":""Float"",""value"":25},""Chamber1ToleranceTempMin"":{""datatype"":""Float"",""value"":23.5},""Chamber1ToleranceTempMax"":{""datatype"":""Float"",""value"":26.5},""Chamber1HumiditySet"":{""datatype"":""Float"",""value"":97.5},""Chamber1ToleranceHumidityMin"":{""datatype"":""Float"",""value"":88.7},""Chamber1ToleranceHumidityMax"":{""datatype"":""Float"",""value"":100},""Chamber1SettlingTimeSet"":{""datatype"":""Short"",""value"":15},""ChamberConveyorVeloSet"":{""datatype"":""Short"",""value"":66},""SensorDriftTempWarnLimit"":{""datatype"":""Float"",""value"":2},""SensorDriftTempAlarmLimit"":{""datatype"":""Float"",""value"":2},""SensorDriftHumWarnLimit"":{""datatype"":""Float"",""value"":3},""SensorDriftHumAlarmLimit"":{""datatype"":""Float"",""value"":5},""DeviationTimeRange"":{""datatype"":""Int16"",""value"":2},""Chamber1FanSet"":{""datatype"":""Short"",""value"":90},""Chamber1FlapSet"":{""datatype"":""Short"",""value"":0}}"
# 2024-01-30 18:37:40.870,KLS_105,Klimaschrank Mod Typ 1 (KLS105),cfRocheCESR.cfpSetup,"{""Info"":{""datatype"":""String"",""value"":{""WorkOrderNumber"":{""datatype"":""String"",""value"":""11140321""},""WorkstepAvo"":{""datatype"":""Short"",""value"":1120000000},""WorkStepNumber"":{""datatype"":""Short"",""value"":47}}},""StationNumber"":{""datatype"":""String"",""value"":""30010230600180213""},""iOperationMode"":{""datatype"":""Short"",""value"":7},""strJobNumber"":{""datatype"":""String"",""value"":""11140321""},""strRecipe"":{""datatype"":""String"",""value"":""""},""strFormat"":{""datatype"":""String"",""value"":""09539158001""},""strProcessStep"":{""datatype"":""String"",""value"":""pO2AuKathode""},""Chamber1TempSet"":{""datatype"":""Float"",""value"":22.5},""Chamber1ToleranceTempMin"":{""datatype"":""Float"",""value"":20},""Chamber1ToleranceTempMax"":{""datatype"":""Float"",""value"":26},""Chamber1HumiditySet"":{""datatype"":""Float"",""value"":45},""Chamber1ToleranceHumidityMin"":{""datatype"":""Float"",""value"":30},""Chamber1ToleranceHumidityMax"":{""datatype"":""Float"",""value"":60},""Chamber1SettlingTimeSet"":{""datatype"":""Short"",""value"":15},""Chamber1ConveyorVeloSet"":{""datatype"":""Short"",""value"":100},""DeviationTimeRange"":{""datatype"":""Int16"",""value"":2},""SensorDriftTempWarnLimit"":{""datatype"":""Float"",""value"":1},""SensorDriftTempAlarmLimit"":{""datatype"":""Float"",""value"":2},""SensorDriftHumWarnLimit"":{""datatype"":""Float"",""value"":3},""SensorDriftHumAlarmLimit"":{""datatype"":""Float"",""value"":5}}"
# Pandas(index=62395, stream='opcuaservice-ASYSMagazinlagerLR1MLR', filename='OPCUAService0.log', severity='INFO', thread='Thread-123', timestampString='2024-02-23 08:13:51.226', serviceName='OPCUA.Service', manufacturer='Asys', machineName=nan, stationDescription=nan, StationNumber=3.00400206001802e+16, methodName=nan, inArgs='{"Info":{"datatype":"String","value":{"WorkOrderNumber":{"datatype":"String","value":"11138069"},"WorkstepAvo":{"datatype":"Short","value":1180000000},"WorkStepNumber":{"datatype":"Short","value":59}}},"StationNumber":{"datatype":"String","value":"30040020600180199"},"TempSet":{"datatype":"Float","value":50},"ToleranceTempMin":{"datatype":"Float","value":48.5},"ToleranceTempMax":{"datatype":"Float","value":51.5},"HumSet":{"datatype":"Float","value":0},"ToleranceHumMin":{"datatype":"Float","value":0},"ToleranceHumMax":{"datatype":"Float","value":0},"SettlingTimeSet":{"datatype":"Int32","value":15},"DeviationTimeRange":{"datatype":"Int32","value":2},"SensorDriftHumAlarmLimit":{"datatype":"Float","value":2},"SensorDriftHumWarnLimit":{"datatype":"Float","value":3},"SensorDriftTempAlarmLimit":{"datatype":"Float","value":2},"SensorDriftTempWarnLimit":{"datatype":"Float","value":1}}', outArgs='50, 48.5, 51.5, 0, 0, 0, 15, 2, 2, 3, 2, 1', returnCode=0.0, customError=nan, duration=nan, message='methodName:in:String --> cfRocheCESR.cfpSetup', _18='{"Info":{"datatype":"String","value":{"WorkOrderNumber":{"datatype":"String","value":"11138069"},"WorkstepAvo":{"datatype":"Short","value":1180000000},"WorkStepNumber":{"datatype":"Short","value":59}}},"StationNumber":{"datatype":"String","value":"30040020600180199"},"TempSet":{"datatype":"Float","value":50},"ToleranceTempMin":{"datatype":"Float","value":48.5},"ToleranceTempMax":{"datatype":"Float","value":51.5},"HumSet":{"datatype":"Float","value":0},"ToleranceHumMin":{"datatype":"Float","value":0},"ToleranceHumMax":{"datatype":"Float","value":0},"SettlingTimeSet":{"datatype":"Int32","value":15},"DeviationTimeRange":{"datatype":"Int32","value":2},"SensorDriftHumAlarmLimit":{"datatype":"Float","value":2},"SensorDriftHumWarnLimit":{"datatype":"Float","value":3},"SensorDriftTempAlarmLimit":{"datatype":"Float","value":2},"SensorDriftTempWarnLimit":{"datatype":"Float","value":1}}')

  def getValue(self, columnName, python_object):
    return  python_object[columnName]['value'] if columnName in python_object else ''
  def getNumValue(self, columnName, python_object):
    return  float(python_object[columnName]['value']) if columnName in python_object else -1.

  def is_valid_json(self, json_string):
      try:
          json.loads(json_string)
          return True
      except ValueError:
          return False

  def ParseJsonData(self, json_data):
    #print('json_data = ' + self.json_data)
    setting = json.loads(self.json_data)
    if (setting):
      #print('setting = ' + setting)
      self.stationNB = self.getValue('StationNumber', setting)
      self.WorkOrderNumber = self.getValue('WorkOrderNumber', setting)
      self.iOperationMode = self.getValue('iOperationMode', setting)
      self.strRecipe = self.getValue('strRecipe', setting)
      self.WorkStepNumber = self.getValue('WorkStepNumber', setting)
      self.strProcessStep = self.getValue('strProcessStep', setting)
      if (self.strRecipe == 'N/A' and self.strProcessStep=='N/A'):
        self.SetupType = 'Basic'
      elif (len(self.strRecipe)>3 and len(self.strProcessStep)>3):
        self.SetupType = 'Workstep'
      else:
        self.SetupType = 'Undefined'

      self.iFanState = self.getNumValue('iFanState', setting)
      self.iFanSpeed = self.getNumValue('iFanSpeed', setting)

      ###
      self.Chamber1TempSet = self.getNumValue('Chamber1TempSet', setting)
      self.Chamber1ToleranceTempMin = self.getNumValue('Chamber1ToleranceTempMin', setting)
      self.Chamber1ToleranceTempMax = self.getNumValue('Chamber1ToleranceTempMax', setting)
      self.Chamber1TempSDMax = self.Chamber1ToleranceTempMax - self.Chamber1TempSet
      self.Chamber1TempSDMin = self.Chamber1TempSet - self.Chamber1ToleranceTempMin
      ###
      self.Chamber1HumiditySet = self.getNumValue('Chamber1HumiditySet', setting)
      self.Chamber1ToleranceHumidityMin = self.getNumValue('Chamber1ToleranceHumidityMin', setting)
      self.Chamber1ToleranceHumidityMax = self.getNumValue('Chamber1ToleranceHumidityMax', setting)
      self.Chamber1HumiditySDMax = self.Chamber1ToleranceHumidityMax - self.Chamber1HumiditySet
      self.Chamber1HumiditySDMin = self.Chamber1HumiditySet - self.Chamber1ToleranceHumidityMin
      ###
      self.Chamber1SettlingTimeSet = self.getNumValue('Chamber1SettlingTimeSet', setting)
      self.Chamber1ConveyorVeloSet = self.getNumValue('Chamber1ConveyorVeloSet', setting)
      self.DeviationTimeRange = self.getNumValue('DeviationTimeRange', setting)
      ###
      self.SensorDriftTempWarnLimit = self.getNumValue('SensorDriftTempWarnLimit', setting)
      self.SensorDriftTempAlarmLimit = self.getNumValue('SensorDriftTempAlarmLimit', setting)
      ###
      self.SensorDriftHumWarnLimit = self.getNumValue('SensorDriftHumWarnLimit', setting)
      self.SensorDriftHumAlarmLimit = self.getNumValue('SensorDriftHumAlarmLimit', setting)

  def Parse(self, row):
      #print(row)
      #print(row[0])
      if isinstance(row[0], str):
      # 3005001060018026505, Temperature, 24.15, c, Humidity, 43.33, %
        self.timestampString = row[0].strip()
        self.machineName = row[1]#.strip() -> exception
        self.stationDescription = row[2] #.strip() -> exception
        self.stationNB = row[3].strip()
        if (len(row)>4):
          self.json_data = row[4]
      else:
      # Pandas(index=62395, stream='opcuaservice-ASYSMagazinlagerLR1MLR', filename='OPCUAService0.log', severity='INFO', thread='Thread-123', timestampString='2024-02-23 08:13:51.226', serviceName='OPCUA.Service', manufacturer='Asys', machineName=nan, stationDescription=nan, StationNumber=3.00400206001802e+16, methodName=nan, inArgs='{"Info":{"datatype":"String","value":{"WorkOrderNumber":{"datatype":"String","value":"11138069"},"WorkstepAvo":{"datatype":"Short","value":1180000000},"WorkStepNumber":{"datatype":"Short","value":59}}},"StationNumber":{"datatype":"String","value":"30040020600180199"},"TempSet":{"datatype":"Float","value":50},"ToleranceTempMin":{"datatype":"Float","value":48.5},"ToleranceTempMax":{"datatype":"Float","value":51.5},"HumSet":{"datatype":"Float","value":0},"ToleranceHumMin":{"datatype":"Float","value":0},"ToleranceHumMax":{"datatype":"Float","value":0},"SettlingTimeSet":{"datatype":"Int32","value":15},"DeviationTimeRange":{"datatype":"Int32","value":2},"SensorDriftHumAlarmLimit":{"datatype":"Float","value":2},"SensorDriftHumWarnLimit":{"datatype":"Float","value":3},"SensorDriftTempAlarmLimit":{"datatype":"Float","value":2},"SensorDriftTempWarnLimit":{"datatype":"Float","value":1}}', outArgs='50, 48.5, 51.5, 0, 0, 0, 15, 2, 2, 3, 2, 1', returnCode=0.0, customError=nan, duration=nan, message='methodName:in:String --> cfRocheCESR.cfpSetup', _18='{"Info":{"datatype":"String","value":{"WorkOrderNumber":{"datatype":"String","value":"11138069"},"WorkstepAvo":{"datatype":"Short","value":1180000000},"WorkStepNumber":{"datatype":"Short","value":59}}},"StationNumber":{"datatype":"String","value":"30040020600180199"},"TempSet":{"datatype":"Float","value":50},"ToleranceTempMin":{"datatype":"Float","value":48.5},"ToleranceTempMax":{"datatype":"Float","value":51.5},"HumSet":{"datatype":"Float","value":0},"ToleranceHumMin":{"datatype":"Float","value":0},"ToleranceHumMax":{"datatype":"Float","value":0},"SettlingTimeSet":{"datatype":"Int32","value":15},"DeviationTimeRange":{"datatype":"Int32","value":2},"SensorDriftHumAlarmLimit":{"datatype":"Float","value":2},"SensorDriftHumWarnLimit":{"datatype":"Float","value":3},"SensorDriftTempAlarmLimit":{"datatype":"Float","value":2},"SensorDriftTempWarnLimit":{"datatype":"Float","value":1}}')
      # Perform integer parsing
        self.index = row[0]
        if (isinstance(row.inArgs, str) and len(row.inArgs)>4):
          self.json_data = row.inArgs

      #print(f'Row = {self.json_data}')
      if (len(self.json_data)>0 and self.json_data[0]=='{'):
        self.ParseJsonData(self.json_data)
      print(f'settings = {self.to_csv()}')


  def isBasicSetup(self):
    #'''returns 1 if it is a basic setup, returns 0 if it is a normal setup.'''
    return True if (len(self.WorkOrderNumber.strip())==0) else False
    #return True if (self.strRecipe == 'N/A' and self.strProcessStep=='N/A') else False

  def isValid(self):

    #if (self.strRecipe.strip() == ''):  print(f'{self.timestampString} {self.machineName} setup= {self.json_data}')

    # if (self.Chamber1TempSet>0):  print(f'{self.timestampString} {self.machineName} Chamber1 Temperature target= {self.Chamber1TempSet} between {self.Chamber1ToleranceTempMin} and {self.Chamber1ToleranceTempMax} within the timeset {self.Chamber1SettlingTimeSet} min and deviation timeRange {self.DeviationTimeRange} min')
    # if (self.Chamber1HumiditySet>0):  print(f'{self.timestampString} {self.machineName} Chamber1 Humidity target= {self.Chamber1HumiditySet} between {self.Chamber1ToleranceHumidityMin} and {self.Chamber1ToleranceHumidityMax} within the timeset {self.Chamber1SettlingTimeSet} min and deviation timeRange {self.DeviationTimeRange} min')
    ## data validation
    return (self.Chamber1TempSet>-1 and self.Chamber1TempSet<101 and self.Chamber1HumiditySet>-1 and self.Chamber1HumiditySet<101)

  @staticmethod
  def get_Columns():
    return ['timestampString', 'machineName', 'stationNumber', 'stationDescription', 'SetupType', 'WorkOrderNumber', 'WorkStepNumber', 'iOperationMode', 'strRecipe', 'strProcessStep'
    , 'iFanState', 'iFanSpeed'
    , 'Chamber1TempSet', 'Chamber1ToleranceTempMin', 'Chamber1ToleranceTempMax'
    , 'Chamber1TempSDMin', 'Chamber1TempSDMax'
    , 'SensorDriftTempWarnLimit', 'SensorDriftTempAlarmLimit'
    , 'Chamber1HumiditySet', 'Chamber1ToleranceHumidityMin', 'Chamber1ToleranceHumidityMax'
    , 'Chamber1HumiditySDMin', 'Chamber1HumiditySDMax'
    , 'SensorDriftHumWarnLimit', 'SensorDriftHumAlarmLimit'
    , 'Chamber1SettlingTimeSet', 'Chamber1ConveyorVeloSet', 'DeviationTimeRange'
    ]

  @staticmethod
  def get_Headers():
    return cfpSetupSettings.get_Columns().join(';')

  def to_csv(self):
    return [self.timestampString, self.machineName, self.stationNB, self.stationDescription, self.SetupType, self.WorkOrderNumber, self.WorkStepNumber, self.iOperationMode, self.strRecipe, self.strProcessStep
    , self.iFanState, self.iFanSpeed
    , self.Chamber1TempSet, self.Chamber1ToleranceTempMin, self.Chamber1ToleranceTempMax
    , self.Chamber1TempSDMin, self.Chamber1TempSDMax
    , self.SensorDriftTempWarnLimit, self.SensorDriftTempAlarmLimit
    , self.Chamber1HumiditySet, self.Chamber1ToleranceHumidityMin, self.Chamber1ToleranceHumidityMax
    , self.Chamber1HumiditySDMin, self.Chamber1HumiditySDMax
    , self.SensorDriftHumWarnLimit, self.SensorDriftHumAlarmLimit
    , self.Chamber1SettlingTimeSet, self.Chamber1ConveyorVeloSet, self.DeviationTimeRange
    ]

  def __str__(self):
    return self.to_csv()


## Setup model verification

In [ ]:
columns = cfpSetupSettings.get_Columns()
i=0
for column in columns:
  print(f'{i} => {column}')
  i+=1

# Data Parsing

## Setup Data Parsing

In [ ]:
def parseSetupDataset(df, colsetupfilename):
  machines = []
  dataNumpy = []
  countRows = 0
  countData = 0
  for row in df.itertuples(index=False):
      # Access the values of each column using row.column_name
      # Perform the desired parsing or data manipulation
      countRows+=1
      # Store the parsed data as needed
      settings = cfpSetupSettings(row=row)
      if (settings.isValid()):
        # Get unique machineName
        machine = settings.machineName
        if machine not in machines:
          machines.append(machine)
        dataNumpy.append(settings.to_csv())
        countData+=1
  # Iterate over the dataframe and annotate the data points
  # for index, row in df.iterrows():
  #     devMin = float(row["Chamber1ToleranceTempMin"])-float(row["Chamber1TempSet"])
  #     row['Chamber1DevMin'] = devMin
  #     devMax = float(row["Chamber1ToleranceTempMax"])-float(row["Chamber1TempSet"])
  #     row['Chamber1DevMax'] = devMax

  # for data in dataNumpy:
  #   print(data)
  print(f'Data parsing has scanned {countRows} rows and selected {countData} datarows.')

  dnp = pd.DataFrame(dataNumpy, columns=cfpSetupSettings.get_Columns())
  dnp.to_csv(colsetupfilename, index=False)  # Save the new DataFrame as a CSV file
  dnp.head()
  getShapeSize('cfpSetupSettings dataNumpy', dnp)

  print(f'cfpSetupSettings columns : {cfpSetupSettings.get_Columns()}')
  return (dnp, machines)

colsetupfilename  = colsetupfilename[:len(colsetupfilename)-7] + 'col2.csv'
(setup_dnp, setup_machines) = parseSetupDataset(setup_df, colsetupfilename)

## Process Data Parsing

In [ ]:
def parseProcessDataset(df):
  machines = []
  dataNumpy = []
  outOfRange = []
  invalidated = []
  countRows = 0
  countData = 0
  for row in df.itertuples(index=False):
      # Access the values of each column using row.column_name
      # Perform the desired parsing or data manipulation
      countRows+=1
      # Store the parsed data as needed
      mes = DewPointMeasurementDetails(row=row)
      if (mes.isOutOfRange() ): # or mes.isValid()==False
        outOfRange.append(mes.to_csv())
      if (mes.isValid()==False):
        invalidated.append(mes.to_csv())
      elif (mes.isValid() and mes.isOutOfRange()==False):
        # Get unique machineName
        machine = mes.machineName
        if machine not in machines:
          machines.append(machine)
        dataNumpy.append(mes.to_csv())
        countData+=1
  # for data in dataNumpy:
  #   print(data)
  print(f'Data parsing has scanned {countRows} rows and selected {countData} datarows.')

  invprocessfilename  = colprocessfilename[:len(colprocessfilename)-7] + 'invalid.csv'
  inv = pd.DataFrame(invalidated, columns=DewPointMeasurementDetails.get_Columns())
  inv.to_csv(invprocessfilename, index=False)  # Save the new DataFrame as a CSV file

  outprocessfilename  = colprocessfilename[:len(colprocessfilename)-7] + 'outOfRange.csv'
  out = pd.DataFrame(outOfRange, columns=DewPointMeasurementDetails.get_Columns())
  out.to_csv(outprocessfilename, index=False)  # Save the new DataFrame as a CSV file

  dewprocessfilename  = colprocessfilename[:len(colprocessfilename)-7] + 'dewCal.csv'
  dnp = pd.DataFrame(dataNumpy, columns=DewPointMeasurementDetails.get_Columns())
  dnp.to_csv(dewprocessfilename, index=False)  # Save the new DataFrame as a CSV file
  print(f'Measurement columns : {DewPointMeasurementDetails.get_Columns()}')
  return (dnp, machines)



In [ ]:
#print(process_df)
(process_dnp, process_machines) = parseProcessDataset(process_df)
process_dnp2 = addDateTimeColumn(process_dnp, 'timestampString', 'datetime')
process_dnp2

# Date Filtering and Formating

In [ ]:
import datetime as dt
from datetime import datetime, timedelta

In [ ]:
def createSimpleCountTable(figsize, rdf, ySeries):
  # Assuming your DataFrame is named 'df'
  table = rdf.groupby(ySeries).size().reset_index(name='count')
  # Sort the table by the 'methodName' column
  if len(ySeries)>0:
    table = table.sort_values(ySeries[0])
  # Create the histogram
  fig, ax = plt.subplots(figsize=figsize) # , layout='constrained'
  ax.axis('off')  # Hide the axis
  ax.table(cellText=table.values, colLabels=table.columns, loc='center')
  # # Show the figure
  plt.show()
  return fig

def createDoubleCountTable(figsize, rdf, ySerie1, ySerie2):
  # Assuming your DataFrame is named 'df'
  table = rdf.groupby([ySerie1, ySerie2]).size().reset_index(name='count')
  table = table.sort_values(by='count', ascending=False)  # Sort values by descending counts
  # Create the histogram
  fig, ax = plt.subplots(figsize=figsize) # , layout='constrained'
  ax.axis('off')  # Hide the axis
  ax.table(cellText=table.values, colLabels=table.columns, loc='center')
  # # Show the figure
  plt.show()
  return fig

def showMinMaxDates(df, legend=''):
  min_date = df['datetime'].min()
  max_date = df['datetime'].max()
  print (f'Legend:{legend}, Min date: {min_date}, Max date: {max_date}')
  result_df = df.groupby('machineName')['datetime'].agg(['first', 'last'])
  fig = createSimpleCountTable((8,4), result_df, ['machineName','first', 'last'])
  return result_df

def showMinMaxInColumn(df, colName, legend=''):
  min_date = df[colName].min()
  max_date = df[colName].max()
  print (f'Legend:{legend}, Min : {min_date}, Max : {max_date}')
  result_df = df.groupby('machineName')[colName].agg(['min', 'max'])
  fig = createSimpleCountTable((8,4), result_df, ['machineName','min', 'max'])
  return result_df


In [ ]:
def getShorterTime(ts):
  # Remove milliseconds and seconds from the rounded datetime
  #shorter_ts = ts.replace(second=0, microsecond=0)

  shorter_ts = ts.replace(microsecond=0) # second=0,

  #print(type(shorter_ts))
  # truncate time string
  #string_ts = np.datetime_as_string(shorter_ts)
  string_ts = str(shorter_ts)
  string_ts  = string_ts[:len(string_ts)]
  str_rounded_max_time = string_ts #+'.00 +01'
  return str_rounded_max_time

def getRoundMinTime(dnp):
  date_format = '%Y-%m-%d %H:%M:%S'
  # Check the length of the X-axis
  len_axis = len(dnp["datetime"])
  if (len_axis>0):
    # Set the minimum value on the X-axis
    min_time = dnp["datetime"].iloc[0]
  else:
    min_time = start_date
  ######
  datetime_str = str(min_time)[:23]
  #datetime_obj = datetime.datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S.%f %z')
  datetime_obj = datetime.strptime(datetime_str,date_format)
  ######
  if (isinstance(min_time, datetime)):
    return min_time
  else:
    return datetime_obj

def getRoundMaxTime(dnp):
  #print (type(max_time))
  date_format = '%Y-%m-%d %H:%M:%S.%f %z'
  date_format = '%Y-%m-%d %H:%M:%S.%f'
  date_format = '%Y-%m-%d %H:%M:%S'
  # Check the length of the axis
  len_axis = len(dnp["datetime"])
  if (len_axis>1):
    # Set the maximum value on the X-axis
    max_time = dnp["datetime"].iloc[-1]
  elif (len_axis>0):
    # Set the maximum value on the X-axis
    max_time = dnp["datetime"].iloc[0]
  else:
    max_time = end_date

  # print (f'Max Time1 = {max_time}')
  # Calculate the number of minutes to the next 5-minute interval
  minutes_to_next_interval = (5 - (max_time.minute % 5)) % 5
  # Round the datetime to the next 5-minute interval
  rounded_max_time = max_time + timedelta(minutes=minutes_to_next_interval)
  # Round the datetime to the nearest 5-minute interval
  #rounded_dt = dt - timedelta(minutes=dt.minute % 5,seconds=dt.second,microseconds=dt.microsecond)
  str_rounded_max_time = getShorterTime(rounded_max_time)
  #rounded_max_time = pd.to_datetime(str_rounded_max_time)
  # print (f'Max Time2 = {str_rounded_max_time}')
  ######
  datetime_str = str(str_rounded_max_time)
  datetime_str = datetime_str[:23]  # Remove the remaining unconverted data
  # print (f'Max Time3 = {datetime_str}')
  datetime_obj = datetime.strptime(datetime_str, date_format)
  #datetime_obj = datetime.strptime(datetime_str[:-3] + datetime_str[-2:], '%Y-%m-%d %H:%M:%S %z')
  #datetime_obj = datetime.datetime.strptime(datetime_str)
  ######
  if (isinstance(rounded_max_time, datetime)):
    return rounded_max_time
  else:
    return datetime_obj



def filterByDate(dnp):
  #######  DATE Filtering
  date_format = '%Y-%m-%d %H:%M:%S.%f %z'
  date_format = '%Y-%m-%d %H:%M:%S'
  timestamps = dnp.iloc[:, 0].values
  times = pd.to_datetime(timestamps).tolist()
  shorttimes = [getShorterTime(shorts) for shorts in times]
  # Assuming you have a pandas DataFrame called 'df' with a column named "datetime"
  if 'datetime' in dnp.columns:
    print ('datetime is already in')
  else:
    dnp.insert(0, "datetimeLegend", shorttimes)
    dnp.insert(0, "datetime", times)
    # Convert datetime values to numerical format
    num_datetime = mdates.date2num(dnp['datetime'])
    dnp.insert(0, 'num_datetime', num_datetime)
    # Set the datetime column as the index
    #dnp.set_index('num_datetime', inplace=True)
    #dnp.set_index('datetime', inplace=True)

  # Get today's date
  #today = dt.date.today()
  today = datetime.now()
  # Set the limits for the x-axis using the desired date and time range:
  # Calculate the start date as 15 days before today
  today_early = today.replace(hour=0, minute=0, second=1, microsecond=0)
  start_date = today_early - dt.timedelta(days=5)
  # Calculate the end date as tomorrow
  end_date = today + dt.timedelta(days=0)

  # Get Data Datetime boundaries
  if (len(times)>0):
    start_date = times[0]
    start_date = times[-1] - dt.timedelta(days=5)
    end_date = times[-1]
    # Print the start and end dates
    print("Start Date:", start_date)
    print("End Date:", end_date)
    dnp = dnp.query('datetime >= @start_date and datetime <= @end_date')

  getShapeSize(f'cfpSetupSettings within dates {start_date} and {end_date}', dnp)
  return (dnp, start_date, end_date)



In [ ]:
print(process_df.iloc(0))

showMinMaxDates(process_df, legend='process_df')
(process_df, start_date, end_date) = filterByDate(process_df)
print('####')
print(setup_dnp.iloc(0))
#showMinMaxDates(setup_dnp, legend='setup_dnp')
(setup_dnp, start_date, end_date) = filterByDate(setup_dnp)
print('####')

In [ ]:
def MachineSortAndFilter(machines):
  #######  MACHINE Filtering

  #values = dnp.iloc[:, 2:-2].values
  #machines = np.unique(list(dnp.iloc[:,1].values))
  #machines = dnp['machineName'].unique()  --> erroneous
  print(f'Number of machines = {len(machines)}')
  print(f'List of machines before cleansing = {machines}')
  #machines = [x for x in machines if not math.isnan(x)]
  #for x in machines:  print(f' {x} is {type(x)}')
  machines = [x for x in machines if isinstance(x, str)]   # -> remove nan machine
  #machines = [x for x in machines if x.startswith('KLS')]
  machines.sort()
  print(f'List of machines after sorting = {machines}')
  print(f'Number of machines = {len(machines)}')
  return machines

print('####')
if(setup_machines):
  machines = MachineSortAndFilter(setup_machines)
print('####')
if(process_machines):
  machines = MachineSortAndFilter(process_machines)
print('####')

# Selected machine
selectedMachine = -1
if (selectedMachine>-1):
  print(f'selectedMachine={machines[selectedMachine]}')
else:
  selectedMachineName = 'ALL'
  print(f'selectedMachine=ALL')

In [ ]:
print(f'==============================================')
if (selectedMachine==-1):
  selectedMachineName = '(ALL)'
else:
  selectedMachineName = machines[selectedMachine]

print(f'Strip Plot for {selectedMachineName} machine')

In [ ]:
def reShapeDataFrame(dnp):
  # Create a new DataFrame with the average, minimum, and maximum values
  new_df = pd.DataFrame({
      'Datetime': dnp['datetime'],
      'Machine': dnp['machineName'],
      'SetupType': dnp['SetupType'],
      'Temperature Setup': dnp['Chamber1TempSet'],
      'Minimum': dnp['Chamber1ToleranceTempMin'],
      'Maximum': dnp['Chamber1ToleranceTempMax'],
      'Chamber1TempSDMin': dnp['Chamber1TempSDMin'],
      'Chamber1TempSDMax': dnp['Chamber1TempSDMax']
  })
  # Sort by MachineName and by datetime with columns "MachineName" and "datetime"
  #new_df = new_df.sort_values(by=["MachineName", df.index])
  #new_df = new_df.sort_values(by=["Machine", 'Datetime'])

  #print(new_df)
  min = new_df.index.min()
  max = new_df.index.max()
  # print(min)
  # print(max)
  return new_df

# Generic Setup Chart

## Generic Define

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Define your custom color palette as a list of RGB tuples
colors = [(0.2, 0.4, 0.6), (0.8, 0.2, 0.4), (0.6, 0.8, 0.2)]

# Create a LinearSegmentedColormap from the color list
custom_palette = LinearSegmentedColormap.from_list('custom_palette', colors)

# Plot a colorbar to visualize the custom palette
# plt.imshow([[i for i in range(10)]], cmap=custom_palette)
# plt.colorbar()
# plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors

# Set the number of colors
num_colors = 40
# Choose a colormap from Matplotlib
colormap = plt.cm.tab20  # You can choose any other colormap as well
# Create a ListedColormap with discrete colors
discrete_cmap = mcolors.ListedColormap(colormap(np.linspace(0, 1, num_colors)))

# Output RGB values for each color
for i, rgb in enumerate(discrete_cmap.colors):
    print(f"Color {i + 1}: RGB = {rgb}")

# Plot a colorbar to visualize the distinct colors
plt.imshow([[i] for i in range(num_colors)], cmap=discrete_cmap, aspect='auto')
plt.colorbar(ticks=range(num_colors))
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('_mpl-gallery')

# make data
np.random.seed(1)
x = np.linspace(0, 8, 16)
y1 = 3 + 4*x/8 + np.random.uniform(0.0, 0.5, len(x))
y2 = 1 + 2*x/8 + np.random.uniform(0.0, 0.5, len(x))

# plot
fig, ax = plt.subplots()

ax.fill_between(x, y1, y2, alpha=.5, linewidth=0)
ax.plot(x, (y1 + y2)/2, linewidth=2)

ax.set(xlim=(0, 8), xticks=np.arange(1, 8),
       ylim=(0, 8), yticks=np.arange(1, 8))
# Set the x-axis limits
# left=None, right=None, emit=True, auto=False, xmin=None, xmax=None
ax.set_xlim(0, 8)
ax.set_ylim(0, 8)
#ax.set_xbound(0, 8)

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

x = np.array([1, 2, 3, 4, 5, 6, 7, 8])
y = np.array([20, 30, 5, 12, 39, 48, 50, 3])

# Create an interpolation function
f = interp1d(x, y, kind='cubic')

# Generate more points for a smoother curve
x_new = np.linspace(x.min(), x.max(), 100)
y_smooth = f(x_new)

plt.plot(x, y, 'o', label='Data Points')
plt.plot(x_new, y_smooth, label='Smooth Curve')

plt.title('Smooth Curve')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()

In [ ]:
def isSameThanInStack(stack=[], row=[]):
    same = False
    if row is not None:
      for elt in stack:
        if ((row['machineName'] == elt['machineName'])and(row['Chamber1TempSet'] == elt['Chamber1TempSet'])): same |=True
    ############
    return same

def getUniqueMachineNames(serie):
  machines = []
  for machine in serie:
    if machine not in machines:
      if isinstance(machine, str):
          machines.append(machine)
  ############
  return machines

def getSerieNames(setup_dnp, process_dnp):
  array1 = []
  if (len(process_dnp)>0): array1 = getUniqueMachineNames(process_dnp['machineName'].unique())
  array2 = []
  if (len(setup_dnp)>0): array2 = getUniqueMachineNames(setup_dnp['machineName'].unique())
  merged_array = array1 + array2
  sorted_array = sorted(merged_array)
  ############
  return sorted_array

def getAllPlotAnnotations(plt):
  # Get all annotations
  annotations = []
  ax = plt.gca()  # Get the current Axes object
  for child in ax.get_children():
      print(type(child))
      if isinstance(child, plt.Annotation):
          annotations.append(child)

  # Print the annotations
  for annotation in annotations:
      print(annotation.get_text())
  ############
  return annotations

def addLegendToWith(plt, setup_dnp):
  #for i in range(len(setup_dnp)):
  #getShapeSize('NewPlot', setup_dnp)
  #nb_plots = len(plt.gca().get_children())
  nb_plots = len(setup_dnp)
  #print(nb_plots)
  sample_size = int(nb_plots * 1.00) # filtre le nombre de fleches
  # Generate sample data indices
  data_indices = np.arange(nb_plots)
  # Select equally spaced data points
  if (sample_size>0):
    selected_data = data_indices[::int(nb_plots / sample_size)]
    # Randomly select 15% of the data points
    random_indices = random.sample(range(nb_plots), sample_size)
    random_indices.sort()

  # Define where are the active labels.
  activeLabels = []
  for i in range(nb_plots):
    if (i in selected_data): activeLabels.append(True)
    else: activeLabels.append(False)
  setup_dnp['activeLabels'] = activeLabels
  #bbox = dict(boxstyle ="round", fc ="0.75")
  i = 0
  prev_Stack = []
  same = False
  # Iterate over the dataframe and annotate the data points
  for index, row in setup_dnp.iterrows():
      # Retrieve the x and y coordinates of each stripplot data point
      stripplot = plt.gca().get_children()[i]

      print(type(stripplot))
      #if (stripplot!=None and isinstance(stripplot, matplotlib.spines.Spine)):
      if (stripplot!=None and type(stripplot) == 'matplotlib.spines.Spine'):
        x_coords = stripplot.get_offsets()[:, 0]
        y_coords = stripplot.get_offsets()[:, 1]
        #print(f'X={x_coords}, Y={y_coords}')
        xtx_coords = (x_coords*(1.1))
        ytx_coords = (y_coords*(1.15))+13*i*0.05

        # Change the alpha value of the hue colors
        hue_colors = stripplot.get_facecolor()
        hue_colors[:, 3] = 0.35 #Same color but lower new_alpha -> 3rd channel

      #print(f'i= {i}, {row["Machine"]}, X={x_coords}, Y={y_coords}, Xt={xtx_coords}, Yt={ytx_coords}')
      #print((index, row['TempSet']))
      #devLeft = row["Minimum"]-row["Temperature Setup"]
      devLeft = row["Chamber1TempSDMin"]
      #devRight = row["Maximum"]-row["Temperature Setup"]
      devRight = row["Chamber1TempSDMax"]
      legend_text = f'{row["machineName"]}, T`C={row["Chamber1TempSet"]} (+{devRight}/{devLeft}) at {row["datetime"]}'
      activeLabel = row['activeLabels']

      # Add text legends next to the data points
      same = isSameThanInStack(prev_Stack, row)
      # if (activeLabel==True):
      if (same==False and activeLabel): # and False <-- Desactivate !!!
        plt.annotate(legend_text, (x_coords, y_coords), size=6, xytext=(xtx_coords, ytx_coords),
                  arrowprops = dict(facecolor=hue_colors, shrink = 0.015)) # bbox = bbox, textcoords="offset points",xytext=(3,-2), ha='center'
      #plt.annotate(row['Machine'], (index, row['TempSet'])) # xytext=(2,2), ha='center', textcoords="offset points"
      i+=1
      if len(prev_Stack) >= 4:  # not more than 4 in neighborhood
          prev_Stack.pop(0)     # Remove the oldest element
      prev_Stack.append(row)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import random

def CreateMasterPlot(figsize, setup_dnp, process_dnp, selectedMachineName, pngfilename):
  # Create a boxplot from the new DataFrame
  #setup_dnp.boxplot()
  print ('Create a process plot for ' + selectedMachineName)
  # Multiple box plots on one Axes
  #fig, ax = plt.subplots()

  # Create a 3D scatter plot
  fig = plt.figure(figsize=figsize)
  ax = fig.add_subplot(1,1,1)
  # ax = fig.add_subplot(111, projection='3d')
  # Format x-axis tick labels
  ax.xaxis.grid(True)
  ax.yaxis.grid(True)

  num_ticks = 11
  # date_fmt = mdates.DateFormatter('%Y-%m-%d %HH:%MM')  # Customize the date format as per your needs
  # ax.xaxis.set_major_formatter(date_fmt)
  ax.xaxis.set_major_locator(plt.MaxNLocator(num_ticks))
  #plt.gca().xaxis.set_major_locator(plt.MaxNLocator(num_ticks))

  # Set y-label
  ax.tick_params(labelrotation=33)
  #ax.legend(loc="upper left")

  # Extract the unique series names and sort them
  series_names = getSerieNames(setup_dnp, process_dnp)

  # Define common palette
  myPalette = ''
  if (selectedMachineName=='(ALL)'):
    myPalette = 'tab20' #   tab20 , palette="Set2"
  else:
    myPalette = 'Set2'

  # Create a process plot for each machine
  if len(process_dnp) > 0:
    sns.lineplot(ax=ax, data=process_dnp, x="datetimeLegend", y="ActualTemperature1", hue="machineName", hue_order=series_names, palette=myPalette)
    sns.lineplot(ax=ax, data=process_dnp, x="datetimeLegend", y="ActualTemperature2", hue="machineName", hue_order=series_names, palette=myPalette, legend=False)
    #sns.scatterplot(ax=ax, data=process_dnp, x="datetimeLegend", y="ActualTemperature1", hue="machineName") # , size=4
    # Draw a violinplot with a narrower bandwidth than the default
    #sns.violinplot(data=process_dnp, x='datetimeLegend', y='ActualTemperature1', bw_adjust=.5, cut=1, linewidth=1, palette="Set3")
    #sns.scatterplot(ax=ax, data=process_dnp, x="datetimeLegend", y="ActualTemperature1", hue="machineName", size=4 )
                    # sizes=(1, 8), linewidth=0, hue_order=clarity_ranking, palette="ch:r=-.2,d=.3_r",

  # Create a Setup plot for each machine
  if (len(setup_dnp)>1):
    ax.fill_between(data=setup_dnp, x='datetimeLegend', y1='Chamber1ToleranceTempMin', y2='Chamber1ToleranceTempMax', alpha=.25, linewidth=0)
    #sns.scatterplot(ax=ax, data=setup_dnp, x='datetimeLegend', y='Chamber1TempSet', hue="machineName", legend='full', size=8)
    sns.stripplot(ax=ax, data=setup_dnp, x='datetimeLegend', y='Chamber1TempSet', hue="machineName", legend='auto', size=8, hue_order=series_names, palette=myPalette)
    #ax.plot(x='Datetime', y='Temperature Setup', hue="Machine", data=setup_dnp, legend='full')
    getAllPlotAnnotations(plt)
    addLegendToWith(plt, setup_dnp)
    # sns.boxenplot(ax=ax, data=setup_dnp, x='datetimeLegend', y='Chamber1ToleranceTempMin', hue="machineName")
    # sns.boxenplot(ax=ax, data=setup_dnp, x='datetimeLegend', y='Chamber1ToleranceTempMax', hue="machineName")

  # # Enable tooltips on hover
  # cursor = mplcursors.cursor(hover=True)
  # cursor.connect("add", lambda sel: sel.annotation.set_text(setup_dnp.loc[sel.target.index]))

  #ax.fill_between(x=setup_dnp['datetimeLegend'], y1=setup_dnp['Chamber1ToleranceTempMin'], y2=setup_dnp['Chamber1ToleranceTempMax'], alpha=.5, linewidth=0, legend='auto')

  # Add a horizontal marker at y=20
  plt.axhline(y=100, color='red', linestyle='--')
  plt.axhline(y=0, color='blue', linestyle='--')
  # Set the maximum value on the Y-axis
  ax.set_ylim(top=105)
  ax.set_ylim(bottom=-5)
  # Set the maximum value on the X-axis
  # plt.gcf().autofmt_xdate()

  rounded_max_time = getRoundMaxTime(setup_dnp)
  print('rounded_max_time:'+str(rounded_max_time))
  # xmin=0.0, xmax=1000
  min_time = getRoundMinTime(setup_dnp)
  print('min_time:'+str(min_time))
  ## ## ## ##
  ax.set_xlim(left=min_time, right=rounded_max_time)
  # plt.gcf().autofmt_xdate()

  # Set the font size of the x-axis legend
  # plt.xlabel(fontsize=9)
  # plt.ylabel(fontsize=9)
  # Set the font size of the x-axis tick labels
  plt.xticks(fontsize=6)
  # Set the font size of the y-axis tick labels
  plt.yticks(fontsize=7)
  # Set the number of x ticks

  # Sort the hue legend in alphabetical order
  # handles, labels = plt.gca().get_legend_handles_labels()
  # labels_sorted, handles_sorted = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
  # # Move the legend outside the chart area

  # Set the legend above the x-axis
  #plt.legend(bbox_to_anchor=(0.5, 1.1), loc='lower center', ncol=5)
  plt.legend(fontsize=6, labelspacing=0.5, ncol=6, loc='lower center')
  # plt.legend(handles_sorted, labels_sorted, bbox_to_anchor=(1.05, 1), loc='upper left')
  #plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

  # Set the legend in the middle place
  plt.title('Temperature Setup per Instrument :' + selectedMachineName)

  # Adjust the layout to prevent overlapping
  plt.tight_layout()
  #plt.colorbar()
  # Display the plot
  plt.show()
  return (fig,ax)

## Execute Setup Chart

In [ ]:
# # prepare data
# new_df = reShapeDataFrame(setup_dnp)
if (setup_dnp.empty==False):
  ### create master plot for each INST
  for machine in machines:
      selectedMachineName = machine
      #selmachine_setup_dnp = None
      selmachine_setup_dnp = setup_dnp[setup_dnp['machineName'] == machine]
      selmachine_process_dnp = process_dnp[process_dnp['machineName'] == machine]
      ###
      pngsetupfilename  = str(colsetupfilename[:len(colsetupfilename)-7]) + '(TH)-Setup'
      (fig,ax) = CreateMasterPlot((12, 8), selmachine_setup_dnp, pd.DataFrame(), selectedMachineName, pngsetupfilename)
      ### Save the plot as PNG
      fig.savefig(pngsetupfilename+f'({selectedMachineName})'+'.png', dpi=120)

      # # create master plot for ALL
      selectedMachineName = 'ALL'
      pngsetupfilename  = str(colsetupfilename[:len(colsetupfilename)-7]) + '(TH)-Setup'
      #(fig,ax) = CreateMasterPlot((24, 12), setup_dnp, pd.DataFrame(), selectedMachineName, pngsetupfilename)
      # # Save the plot as PNG
      #fig.savefig(pngsetupfilename+f'({selectedMachineName})'+'.png', dpi=240)

## Combine Master Setup and Process Data

In [ ]:
# Selected machine
selectedMachine = -1
if (selectedMachine>-1):
  selectedMachineName = machines[selectedMachine]
  print(f'selectedMachine={selectedMachineName}')
else:
  selectedMachineName = 'ALL'
  print(f'selectedMachine=ALL')
if (setup_dnp.empty==False):
  ### create master plot for each INST
  for machine in machines:
      selectedMachineName = machine
      selmachine_setup_dnp = setup_dnp[setup_dnp['machineName'] == machine]
      selmachine_process_dnp = process_dnp[process_dnp['machineName'] == machine]
      ###
      pngsetupfilename  = str(setupfilename[:len(setupfilename)-7]) + '(TH)-combine'
      (fig,ax) = CreateMasterPlot((12, 8), selmachine_setup_dnp, selmachine_process_dnp, selectedMachineName, pngsetupfilename)
      ### Save the plot as PNG
      fig.savefig(pngsetupfilename+f'({selectedMachineName})'+'.png', dpi=320)

# # create master plot for ALL
selectedMachineName = 'ALL'
pngsetupfilename  = str(setupfilename[:len(setupfilename)-7]) + '(TH)-combine'
if (setup_dnp.empty==False):
  (fig,ax) = CreateMasterPlot((24, 12), setup_dnp, process_dnp, selectedMachineName, pngsetupfilename)
  fig.savefig(pngsetupfilename+f'({selectedMachineName})'+'.png', dpi=320)


## Data Statistical Calculation & Enrichments

In [ ]:
import numpy as np

def calculateSTD(data=[], window_size=5):
  num_measurements = len(data)
  standard_deviations = []
  #print(data)
  if num_measurements>0:
    for i in range(num_measurements):
        start_index = (i-window_size) if (i>window_size) else 0
        end_index = i
        # Extract the measurement points within the window
        window_std = 0
        if (end_index>start_index and end_index<num_measurements):
          #print(f' (from {start_index} to {end_index})')
          data_window = data[start_index:end_index]
          #print(data_window)
          # Calculate the standard deviation of the window measurements
          window_std = np.std(data_window)
        # Store the standard deviation value
        if (i>window_size):
          standard_deviations.append(window_std)
        else:
          standard_deviations.append(0)
  ######
  return standard_deviations

# Sort by MachineName and by datetime with columns "MachineName" and "datetime"
#dnp = dnp.sort_values(by=["MachineName", df.index])
#dnp = dnp.sort_values(by=["machineName", 'datetime'])
#print(dnp)

def enrichStatisticsProcessData(dnp, filename):
  # Assuming you have a NumPy dataframe called 'data' with measurement points
  dnp['Temp1_std'] = calculateSTD(dnp['ActualTemperature1'], 10)
  dnp['Humid1_std'] = calculateSTD(dnp['ActualHumidity1'], 10)
  dnp['DewPoint1_std'] = calculateSTD(dnp['DewPoint1'], 10)
  dnp['Temp2_std'] = calculateSTD(dnp['ActualTemperature2'], 10)
  dnp['Humid2_std'] = calculateSTD(dnp['ActualHumidity2'], 10)
  dnp['DewPoint2_std'] = calculateSTD(dnp['DewPoint2'], 10)

  dnp['rActCurrent_X_Axis_STD'] = calculateSTD(dnp['rActCurrent_X_Axis'], 10)
  dnp['rActCurrent_Z_Axis_STD'] = calculateSTD(dnp['rActCurrent_Z_Axis'], 10)

  # Assuming you have a pandas DataFrame called 'df'
  dnp = dnp.sort_index()

  # Assuming you have a NumPy dataframe called 'data'
  column_names = dnp.columns.values.tolist()
  dnp.to_csv(filename, index=False)  # Save the new DataFrame as a CSV file
  print(f'Measurement columns : {column_names}')
  return dnp

stdprocessfilename  = str(processfilename[:len(processfilename)-7]) + 'std.csv'
process_dnp = enrichStatisticsProcessData(process_dnp, stdprocessfilename)

## Generic Function Definition

In [ ]:
#@title Generic SingleScatterPlot
def createSingleScatterPlotPerMachineName(dnp, start_date, end_date, machines, selectedMachine=-1,  dataColumnName='',AxisLegend='', PlotTitle=''):
  # Create scatter plots
  column_index = dnp.columns.get_loc(dataColumnName)
  print(f'Display a SingleScatterPlot chart from column index {column_index} named {dataColumnName}')
  #fig = plt.figure(figsize=(10, 10))  # Increase the figure size as desired
  fig, ax = plt.subplots()
  colors = np.random.rand(50)
  # Create the first plot
  i =0
  # ax = axs[0,0]
  for machine in machines:
    serie_times=dnp[dnp['machineName'] == machine].iloc[:, 0].values
    values_Dw1 = dnp[dnp['machineName'] == machine].iloc[:, column_index].values
    # Set the limits for the x-axis using the desired date and time range:
    ax.set_xlim(start_date, end_date)
    if(i==selectedMachine or selectedMachine==-1):
      ax.scatter(serie_times, values_Dw1, alpha=0.5, label=machine)
    # ax.legend(*[*zip(*{l:h for h,l in zip(*ax.get_legend_handles_labels())}.items())][::-1])

    # Format x-axis tick labels
    #ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))  # Display one label per day
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=6))  # Set the locator to display ticks every xx hours
    #ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Format the labels as 'YYYY-MM-DD'
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))  # Format the labels as 'YYYY-MM-DD HH:MM'
    # Set x-label
    #ax.set_xlabel('Date', rotation=45, ha='left')
    # Set y-label
    ax.set_ylabel(AxisLegend)
    ax.xaxis.set_major_locator(plt.MaxNLocator(10))
    i+=1
    if (i>20): break
  # To specify the number of ticks on both or any single axes
  # plt.locator_params(axis='y', nbins=6)
  # plt.locator_params(axis='x', nbins=10)
  plt.locator_params(nbins=10)
  # Rotate x-axis tick labels for better readability
  plt.xticks(rotation=45)
  # Add legend
  #ax.legend()
  handles, labels = plt.gca().get_legend_handles_labels()
  by_label = dict(zip(labels, handles))
  plt.legend(by_label.values(), by_label.keys(), loc='center', bbox_to_anchor=(0.5, +1.25), ncol=5)

  plt.title(PlotTitle)
  # Adjust the layout to prevent overlapping
  plt.tight_layout()
  # Display the plot
  plt.show()

In [ ]:
#@title Generic Dice Plot
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

def CreateDiceChart(dnp, column_names, legend_names, title, machines, selectedMachine, pngfilename):

  if (selectedMachine==-1):
    selectedMachineName = '(ALL)'
  else:
    selectedMachineName = machines[selectedMachine]

  print(f'==============================================')
  print(f'Scatter Plot for {selectedMachineName} machine')

  # Plot the data
  # plt.xticks(rotation=45)
  fig, axs = plt.subplots(2, 3, figsize=(32, 18), squeeze=False, sharex=True, sharey=False, constrained_layout=True)
  #fig.subplots_adjust(top=0.88) # Leave space on the top of the page for the haupt title
  fig.suptitle(f"{selectedMachineName} Machine Process Data ({title})" , fontsize=28) #

  colors = np.random.rand(50)

  # Create the first plot
  column_index = dnp.columns.get_loc(column_names[0])
  subpt1 = axs[0, 0]
  ax = axs[0,0]
  # Set the limits for the x-axis using the desired date and time range:
  ax.set_xlim(start_date, end_date)
  i = 0
  for machine in machines:
    serie_times= dnp[dnp['machineName'] == machine].iloc[:, 0].values
    values_Dw1 = dnp[dnp['machineName'] == machine].iloc[:, column_index].values
    # Set the limits for the x-axis using the desired date and time range:
    if(i==selectedMachine or selectedMachine==-1):
      ax.scatter(serie_times, values_Dw1, alpha=0.5, label=machine,color=plt.cm.tab20(i))
    # ax.legend(*[*zip(*{l:h for h,l in zip(*ax.get_legend_handles_labels())}.items())][::-1])
    i+=1

  # Format x-axis tick labels
  ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))  # Set the locator to display ticks every xx hours
  ax.xaxis.set_major_locator(plt.MaxNLocator(10))
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))  # Format the labels as 'YYYY-MM-DD HH:MM'
  #ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Format the labels as 'YYYY-MM-DD HH:MM'

  ax.tick_params(labelrotation=45)
  plt.xticks(rotation=45)
  # Set y-label
  ax.legend(loc="upper left")
  ax.set_title(legend_names[0])
  ax.set_ylabel(legend_names[0])

  # Create the second plot
  column_index = dnp.columns.get_loc(column_names[1])
  ax = axs[0,1]
  # Set the limits for the x-axis using the desired date and time range:
  ax.set_xlim(start_date, end_date)
  i = 0
  for machine in machines:
    serie_times= dnp[dnp['machineName'] == machine].iloc[:, 0].values
    values_Dw1 = dnp[dnp['machineName'] == machine].iloc[:, column_index].values
    # Set the limits for the x-axis using the desired date and time range:
    if(i==selectedMachine or selectedMachine==-1):
      ax.scatter(serie_times, values_Dw1, alpha=0.5, label=machine,color=plt.cm.tab20(i))
    # ax.legend(*[*zip(*{l:h for h,l in zip(*ax.get_legend_handles_labels())}.items())][::-1])
    i+=1

  # Format x-axis tick labels
  ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))  # Set the locator to display ticks every xx hours
  ax.xaxis.set_major_locator(plt.MaxNLocator(10))
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))  # Format the labels as 'YYYY-MM-DD HH:MM'
  #ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Format the labels as 'YYYY-MM-DD HH:MM'

  ax.tick_params(labelrotation=45)
  plt.xticks(rotation=45)
  # Set y-label
  ax.legend(loc="upper left")
  ax.set_title(legend_names[1])
  ax.set_ylabel(legend_names[1])

  # Create the third plot
  column_index = dnp.columns.get_loc(column_names[2])
  ax = axs[0,2]
  # Set the limits for the x-axis using the desired date and time range:
  ax.set_xlim(start_date, end_date)
  i = 0
  for machine in machines:
    serie_times= dnp[dnp['machineName'] == machine].iloc[:, 0].values
    values_Dw1 = dnp[dnp['machineName'] == machine].iloc[:, column_index].values
    # Set the limits for the x-axis using the desired date and time range:
    if(i==selectedMachine or selectedMachine==-1):
      ax.scatter(serie_times, values_Dw1, alpha=0.5, label=machine,color=plt.cm.tab20(i))
    # ax.legend(*[*zip(*{l:h for h,l in zip(*ax.get_legend_handles_labels())}.items())][::-1])
    i+=1

  # Format x-axis tick labels
  ax.xaxis.set_major_locator(mdates.HourLocator(interval=12))  # Set the locator to display ticks every xx hours
  ax.xaxis.set_major_locator(plt.MaxNLocator(10))
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))  # Format the labels as 'YYYY-MM-DD HH:MM'
  #ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Format the labels as 'YYYY-MM-DD HH:MM'

  ax.tick_params(labelrotation=45)
  plt.xticks(rotation=45)
  # Set y-label
  ax.legend(loc="upper left")
  ax.set_title(legend_names[2])
  ax.set_ylabel(legend_names[2])

  # Create the fourth plot
  column_index = dnp.columns.get_loc(column_names[3])
  ax = axs[1,0]
  # Set the limits for the x-axis using the desired date and time range:
  ax.set_xlim(start_date, end_date)
  i = 0
  for machine in machines:
    serie_times= dnp[dnp['machineName'] == machine].iloc[:, 0].values
    values_Dw1 = dnp[dnp['machineName'] == machine].iloc[:, column_index].values
    # Set the limits for the x-axis using the desired date and time range:
    if(i==selectedMachine or selectedMachine==-1):
      ax.scatter(serie_times, values_Dw1, alpha=0.5, label=machine,color=plt.cm.tab20(i))
    # ax.legend(*[*zip(*{l:h for h,l in zip(*ax.get_legend_handles_labels())}.items())][::-1])
    i+=1

  # Format x-axis tick labels
  ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))  # Set the locator to display ticks every xx hours
  ax.xaxis.set_major_locator(plt.MaxNLocator(10))
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))  # Format the labels as 'YYYY-MM-DD HH:MM'
  #ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Format the labels as 'YYYY-MM-DD HH:MM'

  ax.tick_params(labelrotation=45)
  plt.xticks(rotation=45)
  # Set y-label
  ax.legend(loc="upper left")
  ax.set_title(legend_names[3])
  ax.set_ylabel(legend_names[3])

  # handles, labels = plt.gca().get_legend_handles_labels()
  # by_label = dict(zip(labels, handles))
  # plt.legend(by_label.values(), by_label.keys(), loc='center', bbox_to_anchor=(0.5, +1.25), ncol=5)

  # Create the fith plot
  column_index = dnp.columns.get_loc(column_names[4])
  ax = axs[1,1]
  ax.set_xlim(start_date, end_date)
  i = 0
  for machine in machines:
    serie_times= dnp[dnp['machineName'] == machine].iloc[:, 0].values
    values_Dw1 = dnp[dnp['machineName'] == machine].iloc[:, column_index].values
    colors = np.random.rand(50)
    # Set the limits for the x-axis using the desired date and time range:
    if(i==selectedMachine or selectedMachine==-1):
      ax.scatter(serie_times, values_Dw1, alpha=0.5, label=machine,color=plt.cm.tab20(i))
    # ax.legend(*[*zip(*{l:h for h,l in zip(*ax.get_legend_handles_labels())}.items())][::-1])
    i+=1

  # Format x-axis tick labels
  ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))  # Set the locator to display ticks every xx hours
  ax.xaxis.set_major_locator(plt.MaxNLocator(10))
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))  # Format the labels as 'YYYY-MM-DD HH:MM'
  #ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Format the labels as 'YYYY-MM-DD HH:MM'
  ax.tick_params(labelrotation=45)
  plt.xticks(rotation=45)
  # Set y-label
  ax.legend(loc="upper left")
  ax.set_title(legend_names[4])
  ax.set_ylabel(legend_names[4])

  # handles, labels = plt.gca().get_legend_handles_labels()
  # by_label = dict(zip(labels, handles))
  # plt.legend(by_label.values(), by_label.keys(), loc='center', bbox_to_anchor=(0.5, +1.25), ncol=5)

  # Create the sixth plot
  column_index = dnp.columns.get_loc(column_names[5])
  ax = axs[1,2]
  # Set the limits for the x-axis using the desired date and time range:
  ax.set_xlim(start_date, end_date)
  i = 0
  for machine in machines:
    serie_times= dnp[dnp['machineName'] == machine].iloc[:, 0].values
    values_Dw1 = dnp[dnp['machineName'] == machine].iloc[:, column_index].values
    colors = np.random.rand(50)
    if(i==selectedMachine or selectedMachine==-1):
      ax.scatter(serie_times, values_Dw1, alpha=0.5, label=machine,color=plt.cm.tab20(i))
    # ax.legend(*[*zip(*{l:h for h,l in zip(*ax.get_legend_handles_labels())}.items())][::-1])
    i+=1

  # Format x-axis tick labels
  ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))  # Set the locator to display ticks every xx hours
  ax.xaxis.set_major_locator(plt.MaxNLocator(10))
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))  # Format the labels as 'YYYY-MM-DD HH:MM'
  #ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Format the labels as 'YYYY-MM-DD HH:MM'
  ax.tick_params(labelrotation=45)
  plt.xticks(rotation=45)
  # Set y-label
  ax.legend(loc="upper left")
  ax.set_title(legend_names[5])
  ax.set_ylabel(legend_names[5])

  # handles, labels = plt.gca().get_legend_handles_labels()
  # by_label = dict(zip(labels, handles))
  # plt.legend(by_label.values(), by_label.keys(), loc='center', bbox_to_anchor=(0.5, +1.25), ncol=5)

  #fig.figure(figsize=(8, 4))  # Width: 8 inches, Height: 4 inches
  # Rotate the x-axis labels for better readability
  # # Adjust the layout to prevent overlapping
  #plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
  #plt.tight_layout()
  #fig.tight_layout() # Or equivalently,  "plt.tight_layout()"

  # Save the plot as PNG
  fig.savefig(pngfilename+f'({selectedMachineName})'+'.png', dpi=320)
  # Display the plots
  plt.show()

In [ ]:
#@title Generic Dual Plot
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from scipy.interpolate import interp1d

# process_dnp, start_date, end_date, columnNames, legendNames, machines, selectedMachine
def createDualScatterPlotPerMachine(dnp, start_date, end_date, machines, selectedMachine, mainTitle, dataColumnNames, dataLegendNames):
  # Create scatter plots
  column_index = dnp.columns.get_loc(dataColumnNames[0])
  print(f'Display a SingleScatterPlot chart from column index {column_index} named {dataColumnNames[0]}')
  # Plot the data
  # plt.xticks(rotation=45)
  fig, axs = plt.subplots(1, 2, figsize=(16, 8), squeeze=False, sharex=True, sharey=False, constrained_layout=True)
  #fig.subplots_adjust(top=0.88) # Leave space on the top of the page for the haupt title.
  fig.suptitle(mainTitle, fontsize=28) #

  colors = np.random.rand(50)

  # Create the first plot
  ax = axs[0,0]
  # Set the limits for the x-axis using the desired date and time range:
  ax.set_xlim(start_date, end_date)
  i = 0
  for machine in machines:
    serie_times= dnp[dnp['machineName'] == machine].iloc[:, 0].values
    values_Dw1 = dnp[dnp['machineName'] == machine].iloc[:, column_index].values
    # Set the limits for the x-axis using the desired date and time range:
    if(i==selectedMachine or selectedMachine==-1):
      #print(f'SelectedMachine={selectedMachine}')
      #print(f'SelectedMachine={machines[i]}')
      ax.scatter(serie_times, values_Dw1, alpha=0.5, label=machines[i], color=plt.cm.tab20(i))
      #ax.legend(*[*zip(*{l:h for h,l in zip(*ax.get_legend_handles_labels())}.items())][::-1])
    i+=1

  # Format x-axis tick labels
  ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))  # Set the locator to display ticks every xx hours
  ax.xaxis.set_major_locator(plt.MaxNLocator(10))
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))  # Format the labels as 'YYYY-MM-DD HH:MM'
  #ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Format the labels as 'YYYY-MM-DD HH:MM'

  ax.tick_params(labelrotation=45)
  plt.xticks(rotation=45)
  # Set y-label
  ax.legend(loc="upper left")
  ax.set_title(dataLegendNames[0])
  ax.set_ylabel(dataLegendNames[0])

  # Create the second plot
  column_index = dnp.columns.get_loc(dataColumnNames[1])
  print(f'Display a SingleScatterPlot chart from column index {column_index} named {dataColumnNames[1]}')
  # Create scatter plots
  ax = axs[0,1]
  # Set the limits for the x-axis using the desired date and time range:
  ax.set_xlim(start_date, end_date)
  i = 0
  for machine in machines:
    serie_times= dnp[dnp['machineName'] == machine].iloc[:, 0].values
    values_Dw1 = dnp[dnp['machineName'] == machine].iloc[:, column_index].values
    # Set the limits for the x-axis using the desired date and time range:
    if(i==selectedMachine or selectedMachine==-1):
      ax.scatter(serie_times, values_Dw1, alpha=0.5, label=machine, color=plt.cm.tab20(i))
      # ax.legend(*[*zip(*{l:h for h,l in zip(*ax.get_legend_handles_labels())}.items())][::-1])
    i+=1

  # Format x-axis tick labels
  ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))  # Set the locator to display ticks every xx hours
  ax.xaxis.set_major_locator(plt.MaxNLocator(10))
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))  # Format the labels as 'YYYY-MM-DD HH:MM'
  #ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Format the labels as 'YYYY-MM-DD HH:MM'

  ax.tick_params(labelrotation=45)
  plt.xticks(rotation=45)
  # Set y-label
  ax.legend(loc="upper left")
  ax.set_title(dataLegendNames[1])
  ax.set_ylabel(dataLegendNames[1])

  # handles, labels = plt.gca().get_legend_handles_labels()
  # by_label = dict(zip(labels, handles))
  # plt.legend(by_label.values(), by_label.keys(), loc='upper left', ncol=5) # , loc='center', bbox_to_anchor=(0, +0.25)

  # Adjust the layout to prevent overlapping
  #plt.tight_layout()
  #fig.tight_layout() # Or equivalently,  "plt.tight_layout()"
  # Display the plots
  plt.show()


# Data Visualisation

In [ ]:
index = machines.index('KLS_310')
selectedMachine = index
print(selectedMachine)

## SubPlots Mechanical Axis

### Mechanical Moves

In [ ]:
(process_dnp2, start_date, end_date) = filterByDate(process_dnp)

selectedMachine = -1
dataColumnNames=['rActCurrent_X_Axis', 'rActCurrent_Z_Axis']
createDualScatterPlotPerMachine(process_dnp,start_date, end_date, machines, selectedMachine, "Machine Process Data (Axis Study)", dataColumnNames, dataColumnNames)

### Mechanical Deviation

In [ ]:
selectedMachine = -1
dataColumnNames=['rActCurrent_X_Axis_STD', 'rActCurrent_Z_Axis_STD']
createDualScatterPlotPerMachine(process_dnp,start_date, end_date, machines, selectedMachine, "Machine Process Data (Axis Deviation)", dataColumnNames, dataColumnNames)

## Generic Dew Dice SubPlots Definition

In [ ]:
# @title display Min Max Temp
showMinMaxInColumn(process_dnp, 'ActualTemperature1', legend='ActualTemperature1')
showMinMaxInColumn(process_dnp, 'ActualTemperature2', legend='ActualTemperature2')
showMinMaxInColumn(process_dnp, 'ActualHumidity1', legend='ActualHumidity1')
showMinMaxInColumn(process_dnp, 'ActualHumidity2', legend='ActualHumidity2')


## Temp1 & Temp2

In [ ]:
selectedMachine = 23
selectedMachine = -1
print(machines)
dataColumnNames=['ActualTemperature1', 'ActualTemperature2']
createDualScatterPlotPerMachine(process_dnp,start_date, end_date, machines, selectedMachine, "Machine Process Data (Temp Study)", dataColumnNames, dataColumnNames)

## Humidity1 & Humidity2

In [ ]:
dataColumnNames=['ActualHumidity1', 'ActualHumidity2']
createDualScatterPlotPerMachine(process_dnp,start_date, end_date, machines, selectedMachine, "Machine Process Data (Humidity Study)", dataColumnNames, dataColumnNames)


## DewPoint 1 & 2

In [ ]:
columnNames = ['DewPoint1', 'DewPoint2']
legendNames = ['Dew Point 1 (degree C)', 'Dew Point 2 (degree C)']
selectedMachine=-1
createDualScatterPlotPerMachine(process_dnp, start_date, end_date, machines, selectedMachine, "Machine Process Data (DewPoint Study)", columnNames, legendNames)

## DewPoint 1 & 2 (Standard Deviation)

In [ ]:
columnNames=['DewPoint1_std', 'DewPoint2_std']
legendNames = ['Dew Point 1 (degree C)', 'Dew Point 2 (degree C)']
createDualScatterPlotPerMachine(process_dnp, start_date, end_date, machines, selectedMachine, "Machine Process Data (DewPoint Deviation)" , columnNames, legendNames)

# Process Data Overview

## *Legends*

In [ ]:
getColumnNames(process_dnp)
print ('############')
columnNames = ['ActualTemperature1', 'ActualHumidity1', 'DewPoint1', 'ActualTemperature2', 'ActualHumidity2', 'DewPoint2']
legendNames = ['Temperature (degree C)  1', 'relative Humidity (%) 1', 'Dew Point 1 (degree C)', 'Temperature (degree C)  2', 'relative Humidity (%) 2', 'Dew Point 2 (degree C)']
for col in range(len(columnNames)): print(f'index ({col}) -> {columnNames[col]}')
selectedMachine = -1
print ('############')
print(machines)

## Visualization of TH Process Data

In [ ]:
selectedMachine = 15
selectedMachine = -1
pngprocessfilename  = str(processfilename[:len(processfilename)-7]) + '(TH)_overview'
CreateDiceChart(process_dnp, columnNames, legendNames, 'Dew Point Study', machines, selectedMachine, pngprocessfilename)

In [ ]:
print(processfilename)
print(pngprocessfilename)

In [ ]:
# ['IST', 'KLS_101', 'KLS_102', 'KLS_103', 'KLS_104', 'KLS_105', 'KLS_106', 'KLS_107', 'KLS_301', 'KLS_303', 'KLS_304', 'KLS_305', 'KLS_306', 'KLS_307', 'KLS_308', 'KLS_310', 'KLS_311', 'KLS_312', 'MLR_1', 'MLR_2', 'MLR_3', 'TPL_DPU', 'TPS_DPU_303', 'TPS_DPU_313', 'TPS_SPU_105']
machine_sel = ['KLS_102', 'KLS_104', 'KLS_106', 'KLS_307', 'KLS_310']
print ('############')
print(machine_sel)

In [ ]:
machine_sel2 = ['KLS_104', 'KLS_301', 'KLS_308', 'KLS_313', 'MLR_1']
print ('############')
print(machine_sel2)

In [ ]:
pngprocessfilename  = str(processfilename[:len(processfilename)-7]) + '(TH)_overview_sel1'
CreateDiceChart(process_dnp, columnNames, legendNames, 'Dew Point Study', machine_sel, selectedMachine, pngprocessfilename)

In [ ]:
pngprocessfilename  = str(processfilename[:len(processfilename)-7]) + '(TH)_overview_sel2'
CreateDiceChart(process_dnp, columnNames, legendNames, 'Dew Point Study', machine_sel2, selectedMachine, pngprocessfilename)

In [ ]:
selectedMachine=0
for machine in machines:
  CreateDiceChart(process_dnp, columnNames, legendNames, 'DewPoint Analysis',  machines, selectedMachine, pngprocessfilename)
  selectedMachine+=1

## Visualization of Standard Deviation

In [ ]:
columnNames = ['Temp1_std', 'Humid1_std', 'DewPoint1_std', 'Temp2_std', 'Humid2_std', 'DewPoint2_std']
legendNames = ['Temperature Deviation (degree C)  1', 'rHumidity Deviation (%) 1', 'Dew Point Deviation 1 (degree C)', 'Temperature Deviation (degree C)  2', 'tHumidity Deviation (%) 2', 'Dew Point Deviation 2 (degree C)']
selectedMachine=23
selectedMachine=-1
pngprocessfilename  = str(processfilename[:len(processfilename)-7]) + '(TH-std)_overview'


In [ ]:
CreateDiceChart(process_dnp, columnNames, legendNames, 'Dew Point Deviation', machines, selectedMachine, pngprocessfilename)

In [ ]:
pngprocessfilename  = str(processfilename[:len(processfilename)-7]) + '(TH-std)_overview_sel1'
CreateDiceChart(process_dnp, columnNames, legendNames, 'Dew Point Deviation', machine_sel, selectedMachine, pngprocessfilename)

In [ ]:
pngprocessfilename  = str(processfilename[:len(processfilename)-7]) + '(TH-std)_overview_sel2'
CreateDiceChart(process_dnp, columnNames, legendNames, 'Dew Point Deviation', machine_sel2, selectedMachine, pngprocessfilename)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
selectedMachine=0
for machine in machines:
  CreateDiceChart(process_dnp, columnNames, legendNames, 'Dew Point Deviation', machines, selectedMachine, pngprocessfilename)
  selectedMachine+=1

### Prepare data

In [ ]:
if (setup_dnp.size>0):
  # create master plot
  (fig,ax) = CreateMasterPlot( (24, 12), setup_dnp, process_dnp, selectedMachineName, pngsetupfilename)
  # Save the plot as PNG
  fig.savefig(pngsetupfilename+f'({selectedMachineName})'+'.png', dpi=320)